# Where the Bugs are

Every time a bug is fixed, developers leave a trace – in the _version database_ when they commit the fix, or in the _bug database_ when they close the bug. In this chapter, we learn how to _mine these repositories_ for past changes and bugs, and how to _map_ them to individual modules and functions, highlighting those project components that have seen most changes and fixes over time.

In [ ]:
from bookutils import YouTubeVideo
YouTubeVideo("Aifq0JOc1Jc")

**Prerequisites**

* You should have read [the chapter on tracking bugs](Tracking.ipynb).

In [ ]:
import bookutils

In [ ]:
import Tracking

## Synopsis
<!-- Automatically generated. Do not edit. -->

To [use the code provided in this chapter](Importing.ipynb), write

```python
>>> from debuggingbook.ChangeCounter import <identifier>
```

and then make use of the following features.


This chapter provides two classes `ChangeCounter` and `FineChangeCounter` that allow to mine and visualize the distribution of changes in a given `git` repository.

`ChangeCounter` is initialized as

```python
change_counter = ChangeCounter(repository)
```
where `repository` is either 

* a _directory_ containing a `git` clone (i.e., it contains a `.git` directory)
* the URL of a `git` repository.

Additional arguments are being passed to the underlying `RepositoryMining` class from the [PyDriller](https://pydriller.readthedocs.io/) Python package. A `filter` keyword argument, if given, is a predicate that takes a modification (from PyDriller) and returns True if it should be included.

In a change counter, all elements in the repository are represented as _nodes_ – tuples $(f_1, f_2, ..., f_n)$ that denote a _hierarchy_: Each $f_i$ is a directory holding $f_{i+1}$, with $f_n$ being the actual file.

A `change_counter` provides a number of attributes. `changes` is a mapping of nodes to the number of changes in that node:

```python
>>> change_counter.changes[('README.md',)]

14
```
The `messages` attribute holds all commit messages related to that node:

```python
>>> change_counter.messages[('README.md',)]

['first commit',
 'Initial import',
 'Adjusted to debuggingbook',
 'New Twitter handle: @Debugging_Book',
 'Doc update',
 'Doc update',
 'Doc update',
 'Doc update',
 'Added link to author homepage',
 'Doc update',
 'Doc update',
 'Doc update',
 'Doc update',
 'Doc update']
```
The `sizes` attribute holds the (last) size of the respective element:

```python
>>> change_counter.sizes[('README.md',)]

13025
```
`FineChangeCounter` acts like `ChangeCounter`, but also retrieves statistics for elements _within_ the respective files; it has been tested for C, Python, and Jupyter Notebooks and should provide sufficient results for programming languages with similar syntax.

The `map()` method of `ChangeCounter` and `FineChangeCounter` produces an interactive tree map that allows to explore the elements of a repository. The redder (darker) a rectangle, the more changes it has seen; the larger a rectangle, the larger its size in bytes.

```python
>>> fine_change_counter.map()
```

<div>                            <div id="8a08ef42-ef66-496f-88ae-baa83aaa844e" class="plotly-graph-div" style="height:525px; width:100%;"></div>            <script type="text/javascript">                require(["plotly"], function(Plotly) {                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("8a08ef42-ef66-496f-88ae-baa83aaa844e")) {                    Plotly.newPlot(                        "8a08ef42-ef66-496f-88ae-baa83aaa844e",                        [{"branchvalues": "total", "hoverinfo": "label+text", "ids": ["/README.md", "/Chapters.makefile", "/LICENSE.md", "/Makefile", "/debuggingbook.bib", "/ipypublish", "/ipypublish_plugins", "/notebooks/.ipynb_checkpoints/index-checkpoint.ipynb", "/notebooks/index.ipynb", "/utils", "/.gitignore", "/full_notebooks/index.ipynb~", "/binder/README.md", "/binder/apt.txt", "/binder/environment.yml", "/binder/postBuild", "/binder/postBuild.template", "/binder/requirements.txt", "/notebooks/99_Appendices.ipynb", "/notebooks/ControlFlow.ipynb", "/notebooks/ExpectError.ipynb", "/notebooks/RailroadDiagrams.ipynb", "/notebooks/Timer.ipynb", "/notebooks/debuggingbook_utils", "/notebooks/fuzzingbook_utils", "/beta/html/PICS", "/full_notebooks/debuggingbook_utils", "/full_notebooks/fuzzingbook_utils", "/html/PICS", "/notebooks/01_Intro.ipynb", "/notebooks/Tours.ipynb", "/html/custom.css", "/html/favicon/README.md", "/html/favicon/android-chrome-144x144.png", "/html/favicon/apple-touch-icon.png", "/html/favicon/browserconfig.xml", "/html/favicon/favicon-16x16.png", "/html/favicon/favicon-32x32.png", "/html/favicon/favicon.ico", "/html/favicon/html_code.html", "/html/favicon/mstile-150x150.png", "/html/favicon/site.webmanifest", "/notebooks/404.ipynb", "/notebooks/404.ipynb/NotFoundError", "/notebooks/Guide_for_Authors.ipynb", "/notebooks/Guide_for_Authors.ipynb/student_example", "/notebooks/Guide_for_Authors.ipynb/Foo", "/notebooks/Importing.ipynb", "/notebooks/Intro_Debugging.ipynb", "/notebooks/Intro_Debugging.ipynb/int_fuzzer", "/notebooks/ReleaseNotes.ipynb", "/notebooks/Template.ipynb", "/notebooks/Template.ipynb/int_fuzzer", "/beta/code/LICENSE.md", "/beta/code/README.md", "/beta/code/__init__.py.in", "/beta/code/debuggingbook_utils", "/beta/code/fuzzingbook_utils", "/beta/code/setup.py.in", "/beta/html/custom.css", "/beta/html/favicon", "/code/LICENSE.md", "/code/README.md", "/code/__init__.py.in", "/code/debuggingbook_utils", "/code/setup.py.in", "/full_notebooks/LICENSE.md", "/full_notebooks/PICS", "/full_notebooks/README.md", "/full_notebooks/custom.css", "/full_notebooks/debuggingbook.bib", "/notebooks/00_Index.ipynb", "/beta/code/bookutils", "/code/bookutils", "/full_notebooks/bookutils", "/fuzzingbook.bib", "/notebooks/Makefile", "/notebooks/bookutils", "/full_notebooks/fuzzingbook.bib", "/notebooks/fuzzingbook.bib", "/PICS", "/notebooks/PICS/wordcloud.png", "/pdf/PICS", "/.gitmodules", "/beta/slides/reveal.js", "/slides/reveal.js", "/notebooks/00_Table_of_Contents.ipynb", "/notebooks/Chapters.ipynb", "/ipypublish_plugins/README.md", "/ipypublish_plugins/html_ipypublish_chapter.py", "/ipypublish_plugins/html_ipypublish_chapter.py/hide_solution", "/ipypublish_plugins/latex_ipypublish_book.py", "/ipypublish_plugins/latex_ipypublish_chapter.py", "/html/favicon/android-chrome-192x192.png", "/html/favicon/android-chrome-512x512.png", "/notebooks/PICS/Icon.png", "/notebooks/PICS/Icons.key", "/notebooks/Debugger.ipynb", "/notebooks/Debugger.ipynb/changed_vars", "/notebooks/Debugger.ipynb/remove_html_markup_traced", "/notebooks/Debugger.ipynb/traceit", "/notebooks/Debugger.ipynb/int_fuzzer", "/notebooks/Debugger.ipynb/debug", "/notebooks/Debugger.ipynb/Tracer", "/notebooks/Tracer.ipynb", "/notebooks/Tracer.ipynb/remove_html_markup_traced", "/notebooks/Tracer.ipynb/traceit", "/notebooks/Tracer.ipynb/Tracer", "/notebooks/Tracer.ipynb/C", "/notebooks/Tracer.ipynb/ConditionalTracer", "/notebooks/Tracer.ipynb/EventTracer", "/notebooks/03_Observing.ipynb", "/notebooks/Intro_Debugging.ipynb/remove_html_markup_with_quote_assert", "/notebooks/Intro_Debugging.ipynb/remove_html_markup_with_proper_quotes", "/notebooks/Intro_Debugging.ipynb/remove_html_markup", "/notebooks/Intro_Debugging.ipynb/remove_html_markup_with_tag_assert", "/notebooks/Debugger.ipynb/Debugger", "/notebooks/Chapters.makefile", "/notebooks/StatisticalDebugger.ipynb", "/notebooks/StatisticalDebugger.ipynb/StatisticalDebugger", "/notebooks/StatisticalDebugger.ipynb/show_covered_lines", "/notebooks/StatisticalDebugger.ipynb/Collector", "/notebooks/StatisticalDebugger.ipynb/DifferenceDebugger", "/notebooks/StatisticalDebugger.ipynb/CoverageCollector", "/notebooks/StatisticalDebugger.ipynb/TarantulaDebugger", "/code/requirements.txt", "/notebooks/requirements.txt", "/requirements.txt", "/notebooks/Intro_Debugging.ipynb/remove_html_markup_fixed", "/notebooks/StatisticalDebugger.ipynb/DiscreteSpectrumDebugger", "/notebooks/StatisticalDebugger.ipynb/list_with_coverage", "/notebooks/StatisticalDebugger.ipynb/test_debugger_middle", "/notebooks/ClassDiagram.ipynb", "/notebooks/PICS/ClassDiagram-synopsis-1.png", "/notebooks/PICS/ClassDiagram-synopsis-1.svg", "/notebooks/PICS/StatisticalDebugger-synopsis-1.png", "/notebooks/PICS/StatisticalDebugger-synopsis-1.svg", "/notebooks/PICS/StatisticalDebugger-synopsis-2.png", "/notebooks/PICS/StatisticalDebugger-synopsis-2.svg", "/notebooks/ClassDiagram.ipynb/public_class_methods", "/notebooks/ClassDiagram.ipynb/D", "/notebooks/ClassDiagram.ipynb/C", "/notebooks/ClassDiagram.ipynb/class_tree", "/notebooks/ClassDiagram.ipynb/display_class_hierarchy", "/notebooks/ClassDiagram.ipynb/A", "/notebooks/ClassDiagram.ipynb/B", "/notebooks/ClassDiagram.ipynb/class_methods", "/notebooks/ClassDiagram.ipynb/doc_class_methods", "/notebooks/ClassDiagram.ipynb/class_hierarchy", "/notebooks/Invariants.ipynb", "/notebooks/Invariants.ipynb/Time", "/notebooks/Invariants.ipynb/test_square_root", "/notebooks/Invariants.ipynb/remove_html_markup", "/notebooks/Invariants.ipynb/some_obscure_function", "/notebooks/Invariants.ipynb/my_own_assert", "/notebooks/Invariants.ipynb/square_root", "/notebooks/Invariants.ipynb/fun", "/notebooks/Invariants.ipynb/RedBlackTree", "/notebooks/Invariants.ipynb/int_fuzzer", "/notebooks/Invariants.ipynb/DynamicMemory", "/.gitattributes", "/notebooks/Assertions.ipynb", "/notebooks/PICS/timetravel_debugger_gui_demo.gif", "/notebooks/Time_Travel_Debugger.ipynb", "/notebooks/Time_Travel_Debugger.ipynb/remove_html_markup", "/notebooks/Time_Travel_Debugger.ipynb/TimeTravelDebugger", "/notebooks/Reducer.ipynb", "/notebooks/Reducer.ipynb/CachingReducer", "/notebooks/Reducer.ipynb/DeltaDebuggingReducer", "/notebooks/Reducer.ipynb/tree_list_to_string", "/notebooks/Reducer.ipynb/CallCollector", "/notebooks/Reducer.ipynb/max_height", "/notebooks/Reducer.ipynb/possible_combinations", "/notebooks/Reducer.ipynb/GrammarReducer", "/notebooks/Reducer.ipynb/number_of_nodes", "/notebooks/Reducer.ipynb/mystery", "/notebooks/Reducer.ipynb/EvalMysteryRunner", "/notebooks/Reducer.ipynb/ZeroDivisionRunner", "/notebooks/Reducer.ipynb/Reducer", "/notebooks/Reducer.ipynb/fuzz", "/notebooks/Assertions.ipynb/remove_html_markup", "/notebooks/DeltaDebugger.ipynb", "/notebooks/DeltaDebugger.ipynb/CachingReducer", "/notebooks/DeltaDebugger.ipynb/generic_test", "/notebooks/DeltaDebugger.ipynb/CallCollector", "/notebooks/DeltaDebugger.ipynb/ddmin", "/notebooks/DeltaDebugger.ipynb/mystery", "/notebooks/DeltaDebugger.ipynb/DeltaDebugger", "/notebooks/DeltaDebugger.ipynb/myeval", "/notebooks/DeltaDebugger.ipynb/Reducer", "/notebooks/DeltaDebugger.ipynb/fuzz", "/notebooks/04_Inputs.ipynb", "/notebooks/DeltaDebugger.ipynb/CachingCallReducer", "/notebooks/ChangeDebugger.ipynb", "/notebooks/ChangeDebugger.ipynb/int_fuzzer", "/notebooks/ChangeDebugger.ipynb/remove_html_markup", "/notebooks/ChangeDebugger.ipynb/write_source", "/notebooks/diff_match_patch/__init__.py", "/notebooks/diff_match_patch/diff_match_patch.py", "/notebooks/diff_match_patch/diff_match_patch.py/diff_match_patch", "/notebooks/diff_match_patch/diff_match_patch.py/patch_obj", "/notebooks/DeltaDebugger.ipynb/compile_and_test_html_markup", "/notebooks/DeltaDebugger.ipynb/NodeReducer", "/notebooks/ChangeDebugger.ipynb/ChangeDebugger", "/notebooks/Draft_Second_Project.ipynb", "/notebooks/Draft_Second_Project.ipynb/DeltaDebuggerReducer", "/notebooks/Draft_Second_Project.ipynb/Parser", "/notebooks/Draft_Second_Project.ipynb/ParserException", "/notebooks/Draft_Second_Project.ipynb/Parser0", "/notebooks/Draft_Second_Project.ipynb/NodeCounter", "/notebooks/Draft_Second_Project.ipynb/Parser2", "/notebooks/Draft_Second_Project.ipynb/Parser3", "/notebooks/Draft_Second_Project.ipynb/Parser4", "/notebooks/Draft_Second_Project.ipynb/Parser5", "/notebooks/Draft_Second_Project.ipynb/MyReducer", "/notebooks/Draft_Second_Project.ipynb/PublicTests", "/notebooks/Draft_Second_Project.ipynb/Parser1", "/notebooks/Draft_Second_Project.ipynb/Reducer", "/notebooks/Slicer.ipynb", "/notebooks/Slicer.ipynb/DataLogger", "/notebooks/Slicer.ipynb/data_access", "/notebooks/Slicer.ipynb/Slicer", "/notebooks/Slicer.ipynb/square_root", "/notebooks/Slicer.ipynb/SaveArgsTransformer", "/notebooks/Slicer.ipynb/tint", "/notebooks/Slicer.ipynb/middle", "/notebooks/Slicer.ipynb/AccessTransformer", "/notebooks/Slicer.ipynb/Instrumenter", "/notebooks/Slicer.ipynb/DataStore", "/notebooks/Reducing_Code.ipynb", "/notebooks/Reducing_Code.ipynb/TestingFramework", "/notebooks/Reducing_Code.ipynb/Parser", "/notebooks/Reducing_Code.ipynb/Test1", "/notebooks/Reducing_Code.ipynb/Test4", "/notebooks/Reducing_Code.ipynb/NodeCounter", "/notebooks/Reducing_Code.ipynb/Parser2", "/notebooks/Reducing_Code.ipynb/MyReducer", "/notebooks/Reducing_Code.ipynb/Test0", "/notebooks/Reducing_Code.ipynb/ReducingDebugger", "/notebooks/Reducing_Code.ipynb/SimpleParserPlus", "/notebooks/Reducing_Code.ipynb/NodeReducer", "/notebooks/Reducing_Code.ipynb/Test5", "/notebooks/Reducing_Code.ipynb/ParserException", "/notebooks/Reducing_Code.ipynb/SimpleParser", "/notebooks/Reducing_Code.ipynb/original", "/notebooks/Reducing_Code.ipynb/Parser4", "/notebooks/Reducing_Code.ipynb/Parser3", "/notebooks/Reducing_Code.ipynb/Parser5", "/notebooks/Reducing_Code.ipynb/foo", "/notebooks/Reducing_Code.ipynb/Test2", "/notebooks/Reducing_Code.ipynb/Test3", "/notebooks/Reducing_Code.ipynb/Parser1", "/notebooks/Slicer.ipynb/Dependencies", "/notebooks/Slicer.ipynb/DataTracker", "/notebooks/PICS/Slicer-synopsis-1.png", "/notebooks/PICS/Slicer-synopsis-1.svg", "/notebooks/PICS/Slicer-synopsis-2.png", "/notebooks/PICS/Slicer-synopsis-2.svg", "/notebooks/Slicer.ipynb/DependencyTracker", "/notebooks/Slicer.ipynb/TrackGetTransformer", "/notebooks/Slicer.ipynb/generator_test", "/notebooks/PICS/ChangeDebugger-synopsis-1.png", "/notebooks/PICS/ChangeDebugger-synopsis-1.svg", "/notebooks/PICS/Debugger-synopsis-1.png", "/notebooks/PICS/Debugger-synopsis-1.svg", "/notebooks/PICS/DeltaDebugger-synopsis-1.png", "/notebooks/PICS/DeltaDebugger-synopsis-1.svg", "/notebooks/PICS/Tracer-synopsis-1.png", "/notebooks/PICS/Tracer-synopsis-1.svg", "/notebooks/Repairer.ipynb", "/notebooks/Repairer.ipynb/middle_tree", "/notebooks/Repairer.ipynb/middle_testcase", "/notebooks/Repairer.ipynb/all_statements", "/notebooks/Repairer.ipynb/middle_passing_testcase", "/notebooks/Repairer.ipynb/RandomNodeMutator", "/notebooks/Repairer.ipynb/middle_test", "/notebooks/Repairer.ipynb/StatementVisitor", "/notebooks/Repairer.ipynb/middle_failing_testcase", "/notebooks/02_Tracking.ipynb", "/notebooks/05_Abstracting.ipynb", "/notebooks/06_Repairing.ipynb", "/code/pycodestyle.cfg", "/notebooks/Repairer.ipynb/StatementMutator", "/notebooks/Repairer.ipynb/random_string", "/notebooks/Repairer.ipynb/ConditionMutator", "/notebooks/Repairer.ipynb/CrossoverOperator", "/notebooks/Repairer.ipynb/evolve_middle", "/notebooks/Repairer.ipynb/Repairer", "/notebooks/PICS/Repairer-synopsis-1.png", "/notebooks/PICS/Repairer-synopsis-1.svg", "/notebooks/PICS/Slicer-synopsis-3.png", "/notebooks/PICS/Slicer-synopsis-3.svg", "/notebooks/Tracking.ipynb", "/notebooks/Tracking.ipynb/all_modifications", "/notebooks/Tracking.ipynb/node_color", "/notebooks/PICS/Tracking-synopsis-1.png", "/notebooks/PICS/Tracking-synopsis-1.svg", "/notebooks/02_Observing.ipynb", "/notebooks/03_Inputs.ipynb", "/notebooks/04_Abstracting.ipynb", "/notebooks/05_Repairing.ipynb", "/notebooks/06_In_the_Large.ipynb", "/notebooks/ChangeExplorer.ipynb", "/notebooks/ChangeExplorer.ipynb/ChangeCounter", "/notebooks/PICS/ChangeExplorer-synopsis-1.png", "/notebooks/PICS/ChangeExplorer-synopsis-1.svg", "/notebooks/Repairer.ipynb/square_root_fixed", "/notebooks/Repairer.ipynb/ConditionVisitor", "/notebooks/Tracking.ipynb/start_webdriver", "/notebooks/Tracking.ipynb/with_ruby", "/notebooks/Tracking.ipynb/start_redmine", "/notebooks/Tracking.ipynb/with_mysql", "/notebooks/Tracking.ipynb/run_redmine", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb/get_arguments", "/notebooks/DynamicInvariants.ipynb/my_sqrt", "/notebooks/DynamicInvariants.ipynb/my_sqrt_with_local_types", "/notebooks/DynamicInvariants.ipynb/EmbeddedInvariantTransformer", "/notebooks/DynamicInvariants.ipynb/my_sqrt_with_type_annotations", "/notebooks/DynamicInvariants.ipynb/instantiate_prop", "/notebooks/DynamicInvariants.ipynb/my_sqrt_with_checked_type_annotations", "/notebooks/DynamicInvariants.ipynb/true_property_instantiations", "/notebooks/DynamicInvariants.ipynb/condition", "/notebooks/DynamicInvariants.ipynb/postcondition", "/notebooks/DynamicInvariants.ipynb/prop_function_text", "/notebooks/DynamicInvariants.ipynb/precondition", "/notebooks/DynamicInvariants.ipynb/InvariantAnnotator", "/notebooks/DynamicInvariants.ipynb/parse_type", "/notebooks/DynamicInvariants.ipynb/simple_call_string", "/notebooks/DynamicInvariants.ipynb/print_sum", "/notebooks/DynamicInvariants.ipynb/Tracker", "/notebooks/DynamicInvariants.ipynb/annotate_function_ast_with_types", "/notebooks/DynamicInvariants.ipynb/CallTracker", "/notebooks/DynamicInvariants.ipynb/sum3", "/notebooks/DynamicInvariants.ipynb/TypeAnnotator", "/notebooks/DynamicInvariants.ipynb/annotate_function_with_types", "/notebooks/DynamicInvariants.ipynb/my_sqrt_with_postcondition", "/notebooks/DynamicInvariants.ipynb/pretty_invariants", "/notebooks/DynamicInvariants.ipynb/PreconditionTransformer", "/notebooks/DynamicInvariants.ipynb/annotate_invariants", "/notebooks/DynamicInvariants.ipynb/instantiate_prop_ast", "/notebooks/DynamicInvariants.ipynb/sum2", "/notebooks/DynamicInvariants.ipynb/TypeTransformer", "/notebooks/DynamicInvariants.ipynb/list_length", "/notebooks/DynamicInvariants.ipynb/remove_first_char", "/notebooks/DynamicInvariants.ipynb/metavars", "/notebooks/DynamicInvariants.ipynb/annotate_function_ast_with_invariants", "/notebooks/DynamicInvariants.ipynb/hello", "/notebooks/DynamicInvariants.ipynb/my_sqrt_with_invariants", "/notebooks/DynamicInvariants.ipynb/my_sqrt_with_precondition", "/notebooks/DynamicInvariants.ipynb/my_sqrt_with_union_type", "/notebooks/DynamicInvariants.ipynb/EmbeddedInvariantAnnotator", "/notebooks/DynamicInvariants.ipynb/InvariantTracker", "/notebooks/DynamicInvariants.ipynb/buggy_my_sqrt_with_postcondition", "/notebooks/DynamicInvariants.ipynb/annotate_types", "/notebooks/DynamicInvariants.ipynb/prop_function", "/notebooks/DynamicInvariants.ipynb/annotate_function_with_invariants", "/notebooks/DynamicInvariants.ipynb/my_sqrt_annotated", "/notebooks/DynamicInvariants.ipynb/type_string", "/notebooks/DynamicInvariants.ipynb/TypeTracker", "/notebooks/DynamicInvariants.ipynb/CallTracer", "/notebooks/Repairing_Code.ipynb", "/notebooks/03_Dependencies.ipynb", "/notebooks/04_Reducing.ipynb", "/notebooks/07_In_the_Large.ipynb", "/notebooks/DDSet.ipynb", "/notebooks/DDSet.ipynb/int_fuzzer", "/notebooks/PICS/ClassDiagram-synopsis-2.png", "/notebooks/PICS/ClassDiagram-synopsis-2.svg", "/notebooks/ChangeCounter.ipynb", "/notebooks/PICS/ChangeCounter-synopsis-1.png", "/notebooks/PICS/ChangeCounter-synopsis-1.svg", "/rendered/PICS", "/notebooks/Repairing_Code.ipynb/debug_and_repair", "/notebooks/Repairing_Code.ipynb/larger_testcase", "/notebooks/Repairing_Code.ipynb/power_failing_testcase", "/notebooks/Repairing_Code.ipynb/larger_test", "/notebooks/Repairing_Code.ipynb/factorial_passing_testcase", "/notebooks/Repairing_Code.ipynb/middle_passing_testcase", "/notebooks/Repairing_Code.ipynb/factorial2_test", "/notebooks/Repairing_Code.ipynb/Tester", "/notebooks/Repairing_Code.ipynb/larger_failing_testcase", "/notebooks/Repairing_Code.ipynb/middle", "/notebooks/Repairing_Code.ipynb/larger", "/notebooks/Repairing_Code.ipynb/middle_failing_testcase", "/notebooks/Repairing_Code.ipynb/middle_assert", "/notebooks/Repairing_Code.ipynb/factorial3", "/notebooks/Repairing_Code.ipynb/power_test", "/notebooks/Repairing_Code.ipynb/Test", "/notebooks/Repairing_Code.ipynb/simple_debug_and_repair", "/notebooks/Repairing_Code.ipynb/larger_passing_testcase", "/notebooks/Repairing_Code.ipynb/middle_testcase", "/notebooks/Repairing_Code.ipynb/factorial1", "/notebooks/Repairing_Code.ipynb/factorial1_test", "/notebooks/Repairing_Code.ipynb/factorial3_test", "/notebooks/Repairing_Code.ipynb/factorial_testcase", "/notebooks/Repairing_Code.ipynb/factorial2", "/notebooks/Repairing_Code.ipynb/power_testcase", "/notebooks/Repairing_Code.ipynb/power_passing_testcase", "/notebooks/Repairing_Code.ipynb/factorial_failing_testcase", "/notebooks/Repairing_Code.ipynb/middle_test", "/notebooks/Repairing_Code.ipynb/power", "/notebooks/DDSet.ipynb/TreeHDDReducer", "/notebooks/DDSet.ipynb/TreeGeneralizer", "/notebooks/PICS/DDSet-synopsis-1.png", "/notebooks/PICS/DDSet-synopsis-1.svg", "/notebooks/PICS/ChangeCounter-synopsis-2.png", "/notebooks/PICS/ChangeCounter-synopsis-2.svg", "/notebooks/Assertions.ipynb/ManagedMemory", "/notebooks/Slicer.ipynb/call_test", "/notebooks/Slicer.ipynb/demo", "/notebooks/DDSet.ipynb/DDSetDebugger", "/notebooks/DDSetDebugger.ipynb", "/notebooks/DDSetDebugger.ipynb/bad_input_tree_reducer", "/notebooks/DDSetDebugger.ipynb/square_root_test", "/notebooks/DDSetDebugger.ipynb/remove_html_markup", "/notebooks/DDSetDebugger.ipynb/TreeGeneralizer", "/notebooks/DDSetDebugger.ipynb/DDSetDebugger", "/notebooks/DDSetDebugger.ipynb/bad_input_tree_hdd_reducer", "/notebooks/DDSetDebugger.ipynb/primes_generator", "/notebooks/DDSetDebugger.ipynb/TreeReducer", "/notebooks/DDSetDebugger.ipynb/TreeHDDReducer", "/notebooks/DDSetDebugger.ipynb/bad_input_tree_mutator", "/notebooks/DDSetDebugger.ipynb/bad_input_tree_generalizer", "/notebooks/DDSetDebugger.ipynb/TreeMutator", "/notebooks/DDSetDebugger.ipynb/middle_test", "/notebooks/DDSetDebugger.ipynb/display_tree", "/notebooks/PICS/DDSetDebugger-synopsis-1.png", "/notebooks/PICS/DDSetDebugger-synopsis-1.svg", "/html/favicon/safari-pinned-tab.svg", "/notebooks/Project_of_your_choice.ipynb", "/code/mypy.ini", "/mypy/mypy.ini", "/mypy/PICS", "/mypy/bookutils", "/notebooks/PerformanceDebugger.ipynb", "/notebooks/ThreadDebugger.ipynb", "/notebooks/PICS/StackInspector-synopsis-1.png", "/notebooks/PICS/StackInspector-synopsis-1.svg", "/notebooks/StackInspector.ipynb", "/notebooks/StackInspector.ipynb/StackInspectorDemo", "/notebooks/StackInspector.ipynb/test", "/notebooks/StackInspector.ipynb/StackInspector", "/", "/notebooks/.ipynb_checkpoints", "/notebooks", "/full_notebooks", "/binder", "/beta/html", "/beta", "/html", "/html/favicon", "/beta/code", "/code", "/notebooks/PICS", "/pdf", "/beta/slides", "/slides", "/notebooks/diff_match_patch", "/rendered", "/mypy"], "labels": ["README.md", "Chapters.makefile", "LICENSE.md", "Makefile", "debuggingbook.bib", "ipypublish", "ipypublish_plugins", "index-checkpoint.ipynb", "index.ipynb", "utils", ".gitignore", "index.ipynb~", "README.md", "apt.txt", "environment.yml", "postBuild", "postBuild.template", "requirements.txt", "99_Appendices.ipynb", "ControlFlow.ipynb", "ExpectError.ipynb", "RailroadDiagrams.ipynb", "Timer.ipynb", "debuggingbook_utils", "fuzzingbook_utils", "PICS", "debuggingbook_utils", "fuzzingbook_utils", "PICS", "01_Intro.ipynb", "Tours.ipynb", "custom.css", "README.md", "android-chrome-144x144.png", "apple-touch-icon.png", "browserconfig.xml", "favicon-16x16.png", "favicon-32x32.png", "favicon.ico", "html_code.html", "mstile-150x150.png", "site.webmanifest", "404.ipynb", "NotFoundError", "Guide_for_Authors.ipynb", "student_example", "Foo", "Importing.ipynb", "Intro_Debugging.ipynb", "int_fuzzer", "ReleaseNotes.ipynb", "Template.ipynb", "int_fuzzer", "LICENSE.md", "README.md", "__init__.py.in", "debuggingbook_utils", "fuzzingbook_utils", "setup.py.in", "custom.css", "favicon", "LICENSE.md", "README.md", "__init__.py.in", "debuggingbook_utils", "setup.py.in", "LICENSE.md", "PICS", "README.md", "custom.css", "debuggingbook.bib", "00_Index.ipynb", "bookutils", "bookutils", "bookutils", "fuzzingbook.bib", "Makefile", "bookutils", "fuzzingbook.bib", "fuzzingbook.bib", "PICS", "wordcloud.png", "PICS", ".gitmodules", "reveal.js", "reveal.js", "00_Table_of_Contents.ipynb", "Chapters.ipynb", "README.md", "html_ipypublish_chapter.py", "hide_solution", "latex_ipypublish_book.py", "latex_ipypublish_chapter.py", "android-chrome-192x192.png", "android-chrome-512x512.png", "Icon.png", "Icons.key", "Debugger.ipynb", "changed_vars", "remove_html_markup_traced", "traceit", "int_fuzzer", "debug", "Tracer", "Tracer.ipynb", "remove_html_markup_traced", "traceit", "Tracer", "C", "ConditionalTracer", "EventTracer", "03_Observing.ipynb", "remove_html_markup_with_quote_assert", "remove_html_markup_with_proper_quotes", "remove_html_markup", "remove_html_markup_with_tag_assert", "Debugger", "Chapters.makefile", "StatisticalDebugger.ipynb", "StatisticalDebugger", "show_covered_lines", "Collector", "DifferenceDebugger", "CoverageCollector", "TarantulaDebugger", "requirements.txt", "requirements.txt", "requirements.txt", "remove_html_markup_fixed", "DiscreteSpectrumDebugger", "list_with_coverage", "test_debugger_middle", "ClassDiagram.ipynb", "ClassDiagram-synopsis-1.png", "ClassDiagram-synopsis-1.svg", "StatisticalDebugger-synopsis-1.png", "StatisticalDebugger-synopsis-1.svg", "StatisticalDebugger-synopsis-2.png", "StatisticalDebugger-synopsis-2.svg", "public_class_methods", "D", "C", "class_tree", "display_class_hierarchy", "A", "B", "class_methods", "doc_class_methods", "class_hierarchy", "Invariants.ipynb", "Time", "test_square_root", "remove_html_markup", "some_obscure_function", "my_own_assert", "square_root", "fun", "RedBlackTree", "int_fuzzer", "DynamicMemory", ".gitattributes", "Assertions.ipynb", "timetravel_debugger_gui_demo.gif", "Time_Travel_Debugger.ipynb", "remove_html_markup", "TimeTravelDebugger", "Reducer.ipynb", "CachingReducer", "DeltaDebuggingReducer", "tree_list_to_string", "CallCollector", "max_height", "possible_combinations", "GrammarReducer", "number_of_nodes", "mystery", "EvalMysteryRunner", "ZeroDivisionRunner", "Reducer", "fuzz", "remove_html_markup", "DeltaDebugger.ipynb", "CachingReducer", "generic_test", "CallCollector", "ddmin", "mystery", "DeltaDebugger", "myeval", "Reducer", "fuzz", "04_Inputs.ipynb", "CachingCallReducer", "ChangeDebugger.ipynb", "int_fuzzer", "remove_html_markup", "write_source", "__init__.py", "diff_match_patch.py", "diff_match_patch", "patch_obj", "compile_and_test_html_markup", "NodeReducer", "ChangeDebugger", "Draft_Second_Project.ipynb", "DeltaDebuggerReducer", "Parser", "ParserException", "Parser0", "NodeCounter", "Parser2", "Parser3", "Parser4", "Parser5", "MyReducer", "PublicTests", "Parser1", "Reducer", "Slicer.ipynb", "DataLogger", "data_access", "Slicer", "square_root", "SaveArgsTransformer", "tint", "middle", "AccessTransformer", "Instrumenter", "DataStore", "Reducing_Code.ipynb", "TestingFramework", "Parser", "Test1", "Test4", "NodeCounter", "Parser2", "MyReducer", "Test0", "ReducingDebugger", "SimpleParserPlus", "NodeReducer", "Test5", "ParserException", "SimpleParser", "original", "Parser4", "Parser3", "Parser5", "foo", "Test2", "Test3", "Parser1", "Dependencies", "DataTracker", "Slicer-synopsis-1.png", "Slicer-synopsis-1.svg", "Slicer-synopsis-2.png", "Slicer-synopsis-2.svg", "DependencyTracker", "TrackGetTransformer", "generator_test", "ChangeDebugger-synopsis-1.png", "ChangeDebugger-synopsis-1.svg", "Debugger-synopsis-1.png", "Debugger-synopsis-1.svg", "DeltaDebugger-synopsis-1.png", "DeltaDebugger-synopsis-1.svg", "Tracer-synopsis-1.png", "Tracer-synopsis-1.svg", "Repairer.ipynb", "middle_tree", "middle_testcase", "all_statements", "middle_passing_testcase", "RandomNodeMutator", "middle_test", "StatementVisitor", "middle_failing_testcase", "02_Tracking.ipynb", "05_Abstracting.ipynb", "06_Repairing.ipynb", "pycodestyle.cfg", "StatementMutator", "random_string", "ConditionMutator", "CrossoverOperator", "evolve_middle", "Repairer", "Repairer-synopsis-1.png", "Repairer-synopsis-1.svg", "Slicer-synopsis-3.png", "Slicer-synopsis-3.svg", "Tracking.ipynb", "all_modifications", "node_color", "Tracking-synopsis-1.png", "Tracking-synopsis-1.svg", "02_Observing.ipynb", "03_Inputs.ipynb", "04_Abstracting.ipynb", "05_Repairing.ipynb", "06_In_the_Large.ipynb", "ChangeExplorer.ipynb", "ChangeCounter", "ChangeExplorer-synopsis-1.png", "ChangeExplorer-synopsis-1.svg", "square_root_fixed", "ConditionVisitor", "start_webdriver", "with_ruby", "start_redmine", "with_mysql", "run_redmine", "DynamicInvariants.ipynb", "get_arguments", "my_sqrt", "my_sqrt_with_local_types", "EmbeddedInvariantTransformer", "my_sqrt_with_type_annotations", "instantiate_prop", "my_sqrt_with_checked_type_annotations", "true_property_instantiations", "condition", "postcondition", "prop_function_text", "precondition", "InvariantAnnotator", "parse_type", "simple_call_string", "print_sum", "Tracker", "annotate_function_ast_with_types", "CallTracker", "sum3", "TypeAnnotator", "annotate_function_with_types", "my_sqrt_with_postcondition", "pretty_invariants", "PreconditionTransformer", "annotate_invariants", "instantiate_prop_ast", "sum2", "TypeTransformer", "list_length", "remove_first_char", "metavars", "annotate_function_ast_with_invariants", "hello", "my_sqrt_with_invariants", "my_sqrt_with_precondition", "my_sqrt_with_union_type", "EmbeddedInvariantAnnotator", "InvariantTracker", "buggy_my_sqrt_with_postcondition", "annotate_types", "prop_function", "annotate_function_with_invariants", "my_sqrt_annotated", "type_string", "TypeTracker", "CallTracer", "Repairing_Code.ipynb", "03_Dependencies.ipynb", "04_Reducing.ipynb", "07_In_the_Large.ipynb", "DDSet.ipynb", "int_fuzzer", "ClassDiagram-synopsis-2.png", "ClassDiagram-synopsis-2.svg", "ChangeCounter.ipynb", "ChangeCounter-synopsis-1.png", "ChangeCounter-synopsis-1.svg", "PICS", "debug_and_repair", "larger_testcase", "power_failing_testcase", "larger_test", "factorial_passing_testcase", "middle_passing_testcase", "factorial2_test", "Tester", "larger_failing_testcase", "middle", "larger", "middle_failing_testcase", "middle_assert", "factorial3", "power_test", "Test", "simple_debug_and_repair", "larger_passing_testcase", "middle_testcase", "factorial1", "factorial1_test", "factorial3_test", "factorial_testcase", "factorial2", "power_testcase", "power_passing_testcase", "factorial_failing_testcase", "middle_test", "power", "TreeHDDReducer", "TreeGeneralizer", "DDSet-synopsis-1.png", "DDSet-synopsis-1.svg", "ChangeCounter-synopsis-2.png", "ChangeCounter-synopsis-2.svg", "ManagedMemory", "call_test", "demo", "DDSetDebugger", "DDSetDebugger.ipynb", "bad_input_tree_reducer", "square_root_test", "remove_html_markup", "TreeGeneralizer", "DDSetDebugger", "bad_input_tree_hdd_reducer", "primes_generator", "TreeReducer", "TreeHDDReducer", "bad_input_tree_mutator", "bad_input_tree_generalizer", "TreeMutator", "middle_test", "display_tree", "DDSetDebugger-synopsis-1.png", "DDSetDebugger-synopsis-1.svg", "safari-pinned-tab.svg", "Project_of_your_choice.ipynb", "mypy.ini", "mypy.ini", "PICS", "bookutils", "PerformanceDebugger.ipynb", "ThreadDebugger.ipynb", "StackInspector-synopsis-1.png", "StackInspector-synopsis-1.svg", "StackInspector.ipynb", "StackInspectorDemo", "test", "StackInspector", "/Users/zeller/Projects/debuggingbook", ".ipynb_checkpoints", "notebooks", "full_notebooks", "binder", "html", "beta", "html", "favicon", "code", "code", "PICS", "pdf", "slides", "slides", "diff_match_patch", "rendered", "mypy"], "marker": {"colors": [14, 37, 1, 1, 1, 1, 1, 3, 14, 1, 1, 2, 1, 3, 4, 11, 2, 3, 3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 3, 4, 10, 2, 2, 5, 2, 5, 5, 5, 2, 5, 3, 6, 1, 17, 1, 1, 7, 51, 1, 4, 8, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 6, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 3, 3, 1, 3, 1, 1, 1, 1, 1, 4, 4, 4, 4, 40, 1, 1, 1, 1, 3, 1, 70, 1, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 2, 1, 83, 3, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 3, 34, 23, 217, 66, 218, 281, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 28, 1, 13, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 71, 1, 1, 3, 1, 1, 2, 1, 1, 1, 1, 1, 25, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 137, 1, 1, 9, 1, 1, 1, 2, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 2, 277, 313, 199, 277, 4, 1, 1, 67, 50, 45, 104, 70, 29, 74, 148, 87, 1, 1, 1, 2, 1, 1, 2, 1, 1, 3, 3, 1, 1, 1, 2, 3, 1, 2, 83, 90, 64, 83, 37, 1, 1, 1, 1, 4, 2, 2, 2, 2, 14, 1, 46, 14, 1, 1, 1, 1, 1, 2, 1, 21, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 2, 3, 2, 17, 1, 4, 5, 21, 7, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 10, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 10, 28, 1, 4, 4, 4, 1, 1, 2, 2, 4, 4, 6, 1, 1, 2, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null], "colorscale": [[0.0, "rgb(255,255,204)"], [0.125, "rgb(255,237,160)"], [0.25, "rgb(254,217,118)"], [0.375, "rgb(254,178,76)"], [0.5, "rgb(253,141,60)"], [0.625, "rgb(252,78,42)"], [0.75, "rgb(227,26,28)"], [0.875, "rgb(189,0,38)"], [1.0, "rgb(128,0,38)"]]}, "parents": ["/", "/", "/", "/", "/", "/", "/", "/notebooks/.ipynb_checkpoints", "/notebooks", "/", "/", "/full_notebooks", "/binder", "/binder", "/binder", "/binder", "/binder", "/binder", "/notebooks", "/notebooks", "/notebooks", "/notebooks", "/notebooks", "/notebooks", "/notebooks", "/beta/html", "/full_notebooks", "/full_notebooks", "/html", "/notebooks", "/notebooks", "/html", "/html/favicon", "/html/favicon", "/html/favicon", "/html/favicon", "/html/favicon", "/html/favicon", "/html/favicon", "/html/favicon", "/html/favicon", "/html/favicon", "/notebooks", "/notebooks/404.ipynb", "/notebooks", "/notebooks/Guide_for_Authors.ipynb", "/notebooks/Guide_for_Authors.ipynb", "/notebooks", "/notebooks", "/notebooks/Intro_Debugging.ipynb", "/notebooks", "/notebooks", "/notebooks/Template.ipynb", "/beta/code", "/beta/code", "/beta/code", "/beta/code", "/beta/code", "/beta/code", "/beta/html", "/beta/html", "/code", "/code", "/code", "/code", "/code", "/full_notebooks", "/full_notebooks", "/full_notebooks", "/full_notebooks", "/full_notebooks", "/notebooks", "/beta/code", "/code", "/full_notebooks", "/", "/notebooks", "/notebooks", "/full_notebooks", "/notebooks", "/", "/notebooks/PICS", "/pdf", "/", "/beta/slides", "/slides", "/notebooks", "/notebooks", "/ipypublish_plugins", "/ipypublish_plugins", "/ipypublish_plugins/html_ipypublish_chapter.py", "/ipypublish_plugins", "/ipypublish_plugins", "/html/favicon", "/html/favicon", "/notebooks/PICS", "/notebooks/PICS", "/notebooks", "/notebooks/Debugger.ipynb", "/notebooks/Debugger.ipynb", "/notebooks/Debugger.ipynb", "/notebooks/Debugger.ipynb", "/notebooks/Debugger.ipynb", "/notebooks/Debugger.ipynb", "/notebooks", "/notebooks/Tracer.ipynb", "/notebooks/Tracer.ipynb", "/notebooks/Tracer.ipynb", "/notebooks/Tracer.ipynb", "/notebooks/Tracer.ipynb", "/notebooks/Tracer.ipynb", "/notebooks", "/notebooks/Intro_Debugging.ipynb", "/notebooks/Intro_Debugging.ipynb", "/notebooks/Intro_Debugging.ipynb", "/notebooks/Intro_Debugging.ipynb", "/notebooks/Debugger.ipynb", "/notebooks", "/notebooks", "/notebooks/StatisticalDebugger.ipynb", "/notebooks/StatisticalDebugger.ipynb", "/notebooks/StatisticalDebugger.ipynb", "/notebooks/StatisticalDebugger.ipynb", "/notebooks/StatisticalDebugger.ipynb", "/notebooks/StatisticalDebugger.ipynb", "/code", "/notebooks", "/", "/notebooks/Intro_Debugging.ipynb", "/notebooks/StatisticalDebugger.ipynb", "/notebooks/StatisticalDebugger.ipynb", "/notebooks/StatisticalDebugger.ipynb", "/notebooks", "/notebooks/PICS", "/notebooks/PICS", "/notebooks/PICS", "/notebooks/PICS", "/notebooks/PICS", "/notebooks/PICS", "/notebooks/ClassDiagram.ipynb", "/notebooks/ClassDiagram.ipynb", "/notebooks/ClassDiagram.ipynb", "/notebooks/ClassDiagram.ipynb", "/notebooks/ClassDiagram.ipynb", "/notebooks/ClassDiagram.ipynb", "/notebooks/ClassDiagram.ipynb", "/notebooks/ClassDiagram.ipynb", "/notebooks/ClassDiagram.ipynb", "/notebooks/ClassDiagram.ipynb", "/notebooks", "/notebooks/Invariants.ipynb", "/notebooks/Invariants.ipynb", "/notebooks/Invariants.ipynb", "/notebooks/Invariants.ipynb", "/notebooks/Invariants.ipynb", "/notebooks/Invariants.ipynb", "/notebooks/Invariants.ipynb", "/notebooks/Invariants.ipynb", "/notebooks/Invariants.ipynb", "/notebooks/Invariants.ipynb", "/", "/notebooks", "/notebooks/PICS", "/notebooks", "/notebooks/Time_Travel_Debugger.ipynb", "/notebooks/Time_Travel_Debugger.ipynb", "/notebooks", "/notebooks/Reducer.ipynb", "/notebooks/Reducer.ipynb", "/notebooks/Reducer.ipynb", "/notebooks/Reducer.ipynb", "/notebooks/Reducer.ipynb", "/notebooks/Reducer.ipynb", "/notebooks/Reducer.ipynb", "/notebooks/Reducer.ipynb", "/notebooks/Reducer.ipynb", "/notebooks/Reducer.ipynb", "/notebooks/Reducer.ipynb", "/notebooks/Reducer.ipynb", "/notebooks/Reducer.ipynb", "/notebooks/Assertions.ipynb", "/notebooks", "/notebooks/DeltaDebugger.ipynb", "/notebooks/DeltaDebugger.ipynb", "/notebooks/DeltaDebugger.ipynb", "/notebooks/DeltaDebugger.ipynb", "/notebooks/DeltaDebugger.ipynb", "/notebooks/DeltaDebugger.ipynb", "/notebooks/DeltaDebugger.ipynb", "/notebooks/DeltaDebugger.ipynb", "/notebooks/DeltaDebugger.ipynb", "/notebooks", "/notebooks/DeltaDebugger.ipynb", "/notebooks", "/notebooks/ChangeDebugger.ipynb", "/notebooks/ChangeDebugger.ipynb", "/notebooks/ChangeDebugger.ipynb", "/notebooks/diff_match_patch", "/notebooks/diff_match_patch", "/notebooks/diff_match_patch/diff_match_patch.py", "/notebooks/diff_match_patch/diff_match_patch.py", "/notebooks/DeltaDebugger.ipynb", "/notebooks/DeltaDebugger.ipynb", "/notebooks/ChangeDebugger.ipynb", "/notebooks", "/notebooks/Draft_Second_Project.ipynb", "/notebooks/Draft_Second_Project.ipynb", "/notebooks/Draft_Second_Project.ipynb", "/notebooks/Draft_Second_Project.ipynb", "/notebooks/Draft_Second_Project.ipynb", "/notebooks/Draft_Second_Project.ipynb", "/notebooks/Draft_Second_Project.ipynb", "/notebooks/Draft_Second_Project.ipynb", "/notebooks/Draft_Second_Project.ipynb", "/notebooks/Draft_Second_Project.ipynb", "/notebooks/Draft_Second_Project.ipynb", "/notebooks/Draft_Second_Project.ipynb", "/notebooks/Draft_Second_Project.ipynb", "/notebooks", "/notebooks/Slicer.ipynb", "/notebooks/Slicer.ipynb", "/notebooks/Slicer.ipynb", "/notebooks/Slicer.ipynb", "/notebooks/Slicer.ipynb", "/notebooks/Slicer.ipynb", "/notebooks/Slicer.ipynb", "/notebooks/Slicer.ipynb", "/notebooks/Slicer.ipynb", "/notebooks/Slicer.ipynb", "/notebooks", "/notebooks/Reducing_Code.ipynb", "/notebooks/Reducing_Code.ipynb", "/notebooks/Reducing_Code.ipynb", "/notebooks/Reducing_Code.ipynb", "/notebooks/Reducing_Code.ipynb", "/notebooks/Reducing_Code.ipynb", "/notebooks/Reducing_Code.ipynb", "/notebooks/Reducing_Code.ipynb", "/notebooks/Reducing_Code.ipynb", "/notebooks/Reducing_Code.ipynb", "/notebooks/Reducing_Code.ipynb", "/notebooks/Reducing_Code.ipynb", "/notebooks/Reducing_Code.ipynb", "/notebooks/Reducing_Code.ipynb", "/notebooks/Reducing_Code.ipynb", "/notebooks/Reducing_Code.ipynb", "/notebooks/Reducing_Code.ipynb", "/notebooks/Reducing_Code.ipynb", "/notebooks/Reducing_Code.ipynb", "/notebooks/Reducing_Code.ipynb", "/notebooks/Reducing_Code.ipynb", "/notebooks/Reducing_Code.ipynb", "/notebooks/Slicer.ipynb", "/notebooks/Slicer.ipynb", "/notebooks/PICS", "/notebooks/PICS", "/notebooks/PICS", "/notebooks/PICS", "/notebooks/Slicer.ipynb", "/notebooks/Slicer.ipynb", "/notebooks/Slicer.ipynb", "/notebooks/PICS", "/notebooks/PICS", "/notebooks/PICS", "/notebooks/PICS", "/notebooks/PICS", "/notebooks/PICS", "/notebooks/PICS", "/notebooks/PICS", "/notebooks", "/notebooks/Repairer.ipynb", "/notebooks/Repairer.ipynb", "/notebooks/Repairer.ipynb", "/notebooks/Repairer.ipynb", "/notebooks/Repairer.ipynb", "/notebooks/Repairer.ipynb", "/notebooks/Repairer.ipynb", "/notebooks/Repairer.ipynb", "/notebooks", "/notebooks", "/notebooks", "/code", "/notebooks/Repairer.ipynb", "/notebooks/Repairer.ipynb", "/notebooks/Repairer.ipynb", "/notebooks/Repairer.ipynb", "/notebooks/Repairer.ipynb", "/notebooks/Repairer.ipynb", "/notebooks/PICS", "/notebooks/PICS", "/notebooks/PICS", "/notebooks/PICS", "/notebooks", "/notebooks/Tracking.ipynb", "/notebooks/Tracking.ipynb", "/notebooks/PICS", "/notebooks/PICS", "/notebooks", "/notebooks", "/notebooks", "/notebooks", "/notebooks", "/notebooks", "/notebooks/ChangeExplorer.ipynb", "/notebooks/PICS", "/notebooks/PICS", "/notebooks/Repairer.ipynb", "/notebooks/Repairer.ipynb", "/notebooks/Tracking.ipynb", "/notebooks/Tracking.ipynb", "/notebooks/Tracking.ipynb", "/notebooks/Tracking.ipynb", "/notebooks/Tracking.ipynb", "/notebooks", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks/DynamicInvariants.ipynb", "/notebooks", "/notebooks", "/notebooks", "/notebooks", "/notebooks", "/notebooks/DDSet.ipynb", "/notebooks/PICS", "/notebooks/PICS", "/notebooks", "/notebooks/PICS", "/notebooks/PICS", "/rendered", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/Repairing_Code.ipynb", "/notebooks/DDSet.ipynb", "/notebooks/DDSet.ipynb", "/notebooks/PICS", "/notebooks/PICS", "/notebooks/PICS", "/notebooks/PICS", "/notebooks/Assertions.ipynb", "/notebooks/Slicer.ipynb", "/notebooks/Slicer.ipynb", "/notebooks/DDSet.ipynb", "/notebooks", "/notebooks/DDSetDebugger.ipynb", "/notebooks/DDSetDebugger.ipynb", "/notebooks/DDSetDebugger.ipynb", "/notebooks/DDSetDebugger.ipynb", "/notebooks/DDSetDebugger.ipynb", "/notebooks/DDSetDebugger.ipynb", "/notebooks/DDSetDebugger.ipynb", "/notebooks/DDSetDebugger.ipynb", "/notebooks/DDSetDebugger.ipynb", "/notebooks/DDSetDebugger.ipynb", "/notebooks/DDSetDebugger.ipynb", "/notebooks/DDSetDebugger.ipynb", "/notebooks/DDSetDebugger.ipynb", "/notebooks/DDSetDebugger.ipynb", "/notebooks/PICS", "/notebooks/PICS", "/html/favicon", "/notebooks", "/code", "/mypy", "/mypy", "/mypy", "/notebooks", "/notebooks", "/notebooks/PICS", "/notebooks/PICS", "/notebooks", "/notebooks/StackInspector.ipynb", "/notebooks/StackInspector.ipynb", "/notebooks/StackInspector.ipynb", null, "/notebooks", "/", "/", "/", "/beta", "/", "/", "/html", "/beta", "/", "/notebooks", "/", "/beta", "/", "/notebooks", "/", "/"], "text": ["14", "37", "1", "1", "1", "1", "1", "3", "14", "1", "1", "2", "1", "3", "4", "11", "2", "3", "3", "1", "1", "1", "1", "1", "1", "2", "1", "1", "1", "3", "4", "10", "2", "2", "5", "2", "5", "5", "5", "2", "5", "3", "6", "1", "17", "1", "1", "7", "51", "1", "4", "8", "3", "1", "1", "1", "1", "1", "1", "1", "1", "1", "2", "2", "1", "6", "1", "1", "1", "1", "1", "3", "1", "1", "1", "1", "1", "1", "1", "1", "1", "2", "1", "1", "3", "3", "1", "3", "1", "1", "1", "1", "1", "4", "4", "4", "4", "40", "1", "1", "1", "1", "3", "1", "70", "1", "1", "1", "1", "3", "1", "1", "1", "3", "1", "1", "2", "1", "83", "3", "1", "1", "1", "1", "2", "1", "1", "1", "1", "2", "1", "1", "3", "34", "23", "217", "66", "218", "281", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "7", "1", "1", "1", "1", "1", "1", "1", "2", "1", "1", "1", "28", "1", "13", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "71", "1", "1", "3", "1", "1", "2", "1", "1", "1", "1", "1", "25", "1", "2", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "137", "1", "1", "9", "1", "1", "1", "2", "1", "1", "1", "8", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "2", "1", "1", "2", "1", "1", "1", "1", "1", "2", "1", "2", "2", "277", "313", "199", "277", "4", "1", "1", "67", "50", "45", "104", "70", "29", "74", "148", "87", "1", "1", "1", "2", "1", "1", "2", "1", "1", "3", "3", "1", "1", "1", "2", "3", "1", "2", "83", "90", "64", "83", "37", "1", "1", "1", "1", "4", "2", "2", "2", "2", "14", "1", "46", "14", "1", "1", "1", "1", "1", "2", "1", "21", "1", "1", "1", "2", "1", "1", "1", "1", "1", "1", "1", "1", "2", "2", "1", "1", "2", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "7", "2", "3", "2", "17", "1", "4", "5", "21", "7", "6", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "2", "1", "1", "1", "2", "1", "1", "1", "10", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "10", "28", "1", "4", "4", "4", "1", "1", "2", "2", "4", "4", "6", "1", "1", "2", null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null], "type": "treemap", "values": [9.474702639106347, 8.220941168281389, 9.930129483482549, 3.1780538303479458, 3.4339872044851463, 3.258096538021482, 41.70577027918862, 9.629642636872369, 9.751093661983862, 3.044522437723423, 3.258096538021482, 0.0, 6.173786103901937, 6.059123195581797, 6.436150368369428, 9.64956257688248, 7.039660349862076, 4.672828834461906, 7.264730177929867, 3.828641396489095, 3.828641396489095, 3.9318256327243257, 3.6888794541139363, 3.828641396489095, 3.828641396489095, 2.772588722239781, 3.4965075614664802, 3.4339872044851463, 2.8903717578961645, 7.346655163176539, 8.657302899400882, 9.572967566074247, 7.091742115095153, 0.0, 8.415824697027949, 5.5093883366279774, 6.833031732786201, 7.045776576879511, 8.416267272826277, 6.3473892096560105, 8.16194579946869, 6.137727054086234, 13.492670973034354, 5.327876168789581, 21.00373908943007, 4.762173934797756, 5.3230099791384085, 8.61286694148452, 48.24033776858043, 5.003946305945459, 7.164720378771857, 14.211181862968168, 5.003946305945459, 3.091042453358316, 3.044522437723423, 3.258096538021482, 3.58351893845611, 3.5263605246161616, 3.1354942159291497, 3.091042453358316, 2.9444389791664403, 7.005789019253503, 7.691656822810547, 4.74493212836325, 3.4965075614664802, 7.138073034044347, 2.6390573296152584, 2.8903717578961645, 2.5649493574615367, 2.9444389791664403, 3.044522437723423, 10.327741702540875, 3.258096538021482, 3.1354942159291497, 3.1354942159291497, 0.0, 2.4849066497880004, 3.6375861597263857, 2.9444389791664403, 2.9444389791664403, 2.70805020110221, 10.920672842508509, 2.8903717578961645, 5.389071729816501, 5.517452896464707, 5.517452896464707, 3.912023005428146, 8.311398278436641, 5.0106352940962555, 15.952900201598315, 6.922643891475888, 8.612503371220562, 8.60337088765729, 8.48446336679332, 9.508146027598816, 10.793824376721568, 12.197501013834398, 59.25163386873523, 5.860786223465865, 5.153291594497779, 7.687538766201629, 5.003946305945459, 7.45298232946546, 8.182279739259018, 49.594459986583594, 5.736572297479192, 5.768320995793772, 8.182279739259018, 4.442651256490317, 7.348587530927593, 6.964135612418245, 7.336285660021297, 6.35088571671474, 6.583409222158765, 7.583756300707112, 6.361302477572996, 9.072915696064511, 3.044522437723423, 70.50150898296374, 8.47907586930311, 6.021023349349527, 5.926926025970411, 6.846943139585379, 6.139884552226255, 5.560681631015528, 5.062595033026967, 5.488937726156687, 5.811140992976701, 4.867534450455582, 7.5126175446745105, 6.037870919922137, 6.269096283706261, 58.40741040933753, 10.971709359880169, 8.969796305981806, 12.652186933226954, 10.197201397219628, 12.331188728553348, 10.028179996493446, 5.099866427824199, 4.394449154672439, 4.189654742026425, 6.489204931325317, 8.006700845440367, 4.189654742026425, 4.77912349311153, 6.421622267806518, 5.0106352940962555, 5.976350909297934, 72.39009724461987, 8.036573409707312, 4.852030263919617, 6.984716320118266, 5.3471075307174685, 4.564348191467836, 6.9440872082295275, 3.8918202981106265, 7.2254814727822945, 5.003946305945459, 8.07961802938984, 4.6913478822291435, 26.4409766519511, 14.227735433940515, 20.76041237548602, 6.202535517187923, 4.110873864173311, 91.27848679807468, 5.968707559985366, 7.035268599281097, 4.787491742782046, 7.277938572945661, 5.262690188904886, 6.1779441140506, 8.863474306170954, 4.948759890378168, 5.529429087511423, 6.0063531596017325, 5.872117789475416, 6.9874902470009905, 5.342334251964811, 6.984716320118266, 89.27513770806189, 5.968707559985366, 5.765191102784844, 8.348537825386098, 6.952728644624869, 5.529429087511423, 8.423321975806166, 3.9889840465642745, 6.7357800142423265, 5.342334251964811, 7.048386408721883, 6.658011045870748, 38.2325882919355, 5.003946305945459, 8.619569258033104, 5.407171771460119, 4.0943445622221, 29.452389289120042, 11.083249628147005, 7.25063551189868, 6.3473892096560105, 7.461065514354283, 7.884199933676039, 92.61499363096868, 7.094234845924755, 6.859614903654202, 4.276666119016055, 6.269096283706261, 6.052089168924417, 6.434546518787453, 7.162397497355718, 7.203405521083095, 6.373319789577012, 5.123963979403259, 7.286191714702382, 6.418364935936212, 6.102558594613569, 127.1385966656026, 6.3080984415095305, 5.313205979041787, 8.659907293615413, 5.8377304471659395, 6.597145701886651, 8.351138607086154, 5.780743515792329, 4.976733742420574, 6.688354713946762, 6.07073772800249, 139.65048254325123, 7.839131648274333, 5.8998973535824915, 4.867534450455582, 4.787491742782046, 6.411818267709897, 5.472270673671475, 5.081404364984463, 4.867534450455582, 7.454141078146678, 5.993961427306569, 6.175867270105761, 4.919980925828125, 4.07753744390572, 7.275864600546533, 7.1569563646156364, 6.946013991099227, 6.794586580876499, 5.560681631015528, 6.331501849893691, 4.787491742782046, 4.787491742782046, 5.407171771460119, 9.570250078424845, 8.363109176033522, 10.853251744467823, 8.93958096672031, 9.155250405658222, 7.715569534520208, 9.431722372670732, 6.587550014824796, 5.53338948872752, 12.011601581466294, 9.75504554717585, 12.06216763247809, 9.756494528795468, 12.203775007981452, 9.974784434418162, 11.905730774931602, 9.670798588378387, 117.55003410985208, 4.454347296253507, 5.147494476813453, 5.736572297479192, 5.641907070938114, 8.17244681834278, 4.718498871295094, 7.085901464365611, 5.545177444479562, 7.030857476116121, 7.428333194190806, 7.205635176410364, 3.6635616461296463, 8.917444732471514, 5.062595033026967, 7.553810852008231, 8.600982717145921, 6.406879986069314, 9.616005460087013, 12.589925890661878, 10.341323040674743, 12.808909376777274, 10.59693468810507, 52.90502614693142, 7.085064293952548, 5.153291594497779, 11.348934864860997, 8.881141619321468, 7.533158807455563, 7.049254841255837, 7.036148493750536, 7.02108396428914, 7.035268599281097, 19.23401513892462, 8.23403420769204, 11.685970796983787, 9.30328411761892, 5.932245187448011, 7.126890808898808, 7.15305163493748, 5.123963979403259, 5.517452896464707, 7.007600613951853, 4.727387818712341, 277.64431466649853, 5.802118375377063, 7.013915474810528, 5.976350909297934, 7.387090235656757, 5.19295685089021, 5.755742213586912, 5.236441962829949, 6.70196036600254, 7.285506548522785, 5.3471075307174685, 4.852030263919617, 5.44673737166631, 8.130942302231878, 5.84354441703136, 6.180016653652572, 3.9889840465642745, 0.0, 7.097548850614793, 7.816416983691801, 4.02535169073515, 6.796823718274855, 5.883322388488279, 4.290459441148391, 5.3706380281276624, 7.226936018493289, 6.021023349349527, 6.598509028614515, 5.713732805509369, 7.427144133408616, 5.1647859739235145, 4.867534450455582, 5.869296913133774, 5.4638318050256105, 4.110873864173311, 5.755742213586912, 4.276666119016055, 4.430816798843313, 7.1569563646156364, 7.028201432058005, 4.442651256490317, 5.958424693029782, 4.418840607796598, 5.811140992976701, 4.356708826689592, 4.276666119016055, 4.04305126783455, 7.9287663216266955, 166.56794774875826, 7.266827347520591, 7.3938782901077555, 7.315218389752975, 39.88582711704645, 5.003946305945459, 10.55843949396453, 8.48528964240323, 11.08328036286033, 11.685970796983787, 9.367429587218957, 3.1354942159291497, 6.839476438228843, 4.890349128221754, 5.497168225293202, 4.90527477843843, 6.720220155135295, 5.641907070938114, 4.653960350157523, 6.38856140554563, 5.5093883366279774, 5.739792912179234, 4.564348191467836, 5.545177444479562, 4.330733340286331, 4.919980925828125, 4.532599493153256, 6.732210706467206, 7.3833681469923835, 5.60947179518496, 5.147494476813453, 4.882801922586371, 4.653960350157523, 4.653960350157523, 4.543294782270004, 4.875197323201151, 4.882801922586371, 5.598421958998375, 6.621405651764134, 4.718498871295094, 4.867534450455582, 7.113956109566034, 8.0774471493312, 0.0, 9.735128070758227, 11.685970796983787, 9.332380997885208, 7.854381210652365, 6.169610732491456, 4.770684624465665, 8.356319965828153, 99.07955130529751, 5.1298987149230735, 4.276666119016055, 6.375024819828097, 8.511778558714738, 8.356319965828153, 5.267858159063328, 6.131226489483141, 7.498869733976931, 7.113956109566034, 4.762173934797756, 5.153291594497779, 7.71333788887187, 4.770684624465665, 6.638567789166521, 12.24399443852873, 10.06296686708145, 7.857867559331803, 8.810161268297255, 6.818924065275521, 6.939253946041508, 2.8903717578961645, 3.1354942159291497, 8.914357274485015, 8.89795568688403, 11.289631902404894, 8.853379672927634, 28.967224987887434, 5.697093486505405, 4.859812404361672, 8.614138397472717, 2942.9266986971174, 9.62964263687238, 2558.9363724344043, 27.093767822910067, 40.03111142905967, 8.808070154764547, 37.222654697355445, 102.27290907214855, 89.80956974817802, 22.89713164612615, 48.75753352629947, 438.1510537974973, 2.8903717578961676, 5.517452896464714, 5.517452896464714, 33.546733851342175, 3.1354942159291532, 12.965119919866837]}],                        {"margin": {"b": 0, "l": 0, "r": 0, "t": 30}, "template": {"data": {"bar": [{"error_x": {"color": "#2a3f5f"}, "error_y": {"color": "#2a3f5f"}, "marker": {"line": {"color": "#E5ECF6", "width": 0.5}}, "type": "bar"}], "barpolar": [{"marker": {"line": {"color": "#E5ECF6", "width": 0.5}}, "type": "barpolar"}], "carpet": [{"aaxis": {"endlinecolor": "#2a3f5f", "gridcolor": "white", "linecolor": "white", "minorgridcolor": "white", "startlinecolor": "#2a3f5f"}, "baxis": {"endlinecolor": "#2a3f5f", "gridcolor": "white", "linecolor": "white", "minorgridcolor": "white", "startlinecolor": "#2a3f5f"}, "type": "carpet"}], "choropleth": [{"colorbar": {"outlinewidth": 0, "ticks": ""}, "type": "choropleth"}], "contour": [{"colorbar": {"outlinewidth": 0, "ticks": ""}, "colorscale": [[0.0, "#0d0887"], [0.1111111111111111, "#46039f"], [0.2222222222222222, "#7201a8"], [0.3333333333333333, "#9c179e"], [0.4444444444444444, "#bd3786"], [0.5555555555555556, "#d8576b"], [0.6666666666666666, "#ed7953"], [0.7777777777777778, "#fb9f3a"], [0.8888888888888888, "#fdca26"], [1.0, "#f0f921"]], "type": "contour"}], "contourcarpet": [{"colorbar": {"outlinewidth": 0, "ticks": ""}, "type": "contourcarpet"}], "heatmap": [{"colorbar": {"outlinewidth": 0, "ticks": ""}, "colorscale": [[0.0, "#0d0887"], [0.1111111111111111, "#46039f"], [0.2222222222222222, "#7201a8"], [0.3333333333333333, "#9c179e"], [0.4444444444444444, "#bd3786"], [0.5555555555555556, "#d8576b"], [0.6666666666666666, "#ed7953"], [0.7777777777777778, "#fb9f3a"], [0.8888888888888888, "#fdca26"], [1.0, "#f0f921"]], "type": "heatmap"}], "heatmapgl": [{"colorbar": {"outlinewidth": 0, "ticks": ""}, "colorscale": [[0.0, "#0d0887"], [0.1111111111111111, "#46039f"], [0.2222222222222222, "#7201a8"], [0.3333333333333333, "#9c179e"], [0.4444444444444444, "#bd3786"], [0.5555555555555556, "#d8576b"], [0.6666666666666666, "#ed7953"], [0.7777777777777778, "#fb9f3a"], [0.8888888888888888, "#fdca26"], [1.0, "#f0f921"]], "type": "heatmapgl"}], "histogram": [{"marker": {"colorbar": {"outlinewidth": 0, "ticks": ""}}, "type": "histogram"}], "histogram2d": [{"colorbar": {"outlinewidth": 0, "ticks": ""}, "colorscale": [[0.0, "#0d0887"], [0.1111111111111111, "#46039f"], [0.2222222222222222, "#7201a8"], [0.3333333333333333, "#9c179e"], [0.4444444444444444, "#bd3786"], [0.5555555555555556, "#d8576b"], [0.6666666666666666, "#ed7953"], [0.7777777777777778, "#fb9f3a"], [0.8888888888888888, "#fdca26"], [1.0, "#f0f921"]], "type": "histogram2d"}], "histogram2dcontour": [{"colorbar": {"outlinewidth": 0, "ticks": ""}, "colorscale": [[0.0, "#0d0887"], [0.1111111111111111, "#46039f"], [0.2222222222222222, "#7201a8"], [0.3333333333333333, "#9c179e"], [0.4444444444444444, "#bd3786"], [0.5555555555555556, "#d8576b"], [0.6666666666666666, "#ed7953"], [0.7777777777777778, "#fb9f3a"], [0.8888888888888888, "#fdca26"], [1.0, "#f0f921"]], "type": "histogram2dcontour"}], "mesh3d": [{"colorbar": {"outlinewidth": 0, "ticks": ""}, "type": "mesh3d"}], "parcoords": [{"line": {"colorbar": {"outlinewidth": 0, "ticks": ""}}, "type": "parcoords"}], "pie": [{"automargin": true, "type": "pie"}], "scatter": [{"marker": {"colorbar": {"outlinewidth": 0, "ticks": ""}}, "type": "scatter"}], "scatter3d": [{"line": {"colorbar": {"outlinewidth": 0, "ticks": ""}}, "marker": {"colorbar": {"outlinewidth": 0, "ticks": ""}}, "type": "scatter3d"}], "scattercarpet": [{"marker": {"colorbar": {"outlinewidth": 0, "ticks": ""}}, "type": "scattercarpet"}], "scattergeo": [{"marker": {"colorbar": {"outlinewidth": 0, "ticks": ""}}, "type": "scattergeo"}], "scattergl": [{"marker": {"colorbar": {"outlinewidth": 0, "ticks": ""}}, "type": "scattergl"}], "scattermapbox": [{"marker": {"colorbar": {"outlinewidth": 0, "ticks": ""}}, "type": "scattermapbox"}], "scatterpolar": [{"marker": {"colorbar": {"outlinewidth": 0, "ticks": ""}}, "type": "scatterpolar"}], "scatterpolargl": [{"marker": {"colorbar": {"outlinewidth": 0, "ticks": ""}}, "type": "scatterpolargl"}], "scatterternary": [{"marker": {"colorbar": {"outlinewidth": 0, "ticks": ""}}, "type": "scatterternary"}], "surface": [{"colorbar": {"outlinewidth": 0, "ticks": ""}, "colorscale": [[0.0, "#0d0887"], [0.1111111111111111, "#46039f"], [0.2222222222222222, "#7201a8"], [0.3333333333333333, "#9c179e"], [0.4444444444444444, "#bd3786"], [0.5555555555555556, "#d8576b"], [0.6666666666666666, "#ed7953"], [0.7777777777777778, "#fb9f3a"], [0.8888888888888888, "#fdca26"], [1.0, "#f0f921"]], "type": "surface"}], "table": [{"cells": {"fill": {"color": "#EBF0F8"}, "line": {"color": "white"}}, "header": {"fill": {"color": "#C8D4E3"}, "line": {"color": "white"}}, "type": "table"}]}, "layout": {"annotationdefaults": {"arrowcolor": "#2a3f5f", "arrowhead": 0, "arrowwidth": 1}, "autotypenumbers": "strict", "coloraxis": {"colorbar": {"outlinewidth": 0, "ticks": ""}}, "colorscale": {"diverging": [[0, "#8e0152"], [0.1, "#c51b7d"], [0.2, "#de77ae"], [0.3, "#f1b6da"], [0.4, "#fde0ef"], [0.5, "#f7f7f7"], [0.6, "#e6f5d0"], [0.7, "#b8e186"], [0.8, "#7fbc41"], [0.9, "#4d9221"], [1, "#276419"]], "sequential": [[0.0, "#0d0887"], [0.1111111111111111, "#46039f"], [0.2222222222222222, "#7201a8"], [0.3333333333333333, "#9c179e"], [0.4444444444444444, "#bd3786"], [0.5555555555555556, "#d8576b"], [0.6666666666666666, "#ed7953"], [0.7777777777777778, "#fb9f3a"], [0.8888888888888888, "#fdca26"], [1.0, "#f0f921"]], "sequentialminus": [[0.0, "#0d0887"], [0.1111111111111111, "#46039f"], [0.2222222222222222, "#7201a8"], [0.3333333333333333, "#9c179e"], [0.4444444444444444, "#bd3786"], [0.5555555555555556, "#d8576b"], [0.6666666666666666, "#ed7953"], [0.7777777777777778, "#fb9f3a"], [0.8888888888888888, "#fdca26"], [1.0, "#f0f921"]]}, "colorway": ["#636efa", "#EF553B", "#00cc96", "#ab63fa", "#FFA15A", "#19d3f3", "#FF6692", "#B6E880", "#FF97FF", "#FECB52"], "font": {"color": "#2a3f5f"}, "geo": {"bgcolor": "white", "lakecolor": "white", "landcolor": "#E5ECF6", "showlakes": true, "showland": true, "subunitcolor": "white"}, "hoverlabel": {"align": "left"}, "hovermode": "closest", "mapbox": {"style": "light"}, "paper_bgcolor": "white", "plot_bgcolor": "#E5ECF6", "polar": {"angularaxis": {"gridcolor": "white", "linecolor": "white", "ticks": ""}, "bgcolor": "#E5ECF6", "radialaxis": {"gridcolor": "white", "linecolor": "white", "ticks": ""}}, "scene": {"xaxis": {"backgroundcolor": "#E5ECF6", "gridcolor": "white", "gridwidth": 2, "linecolor": "white", "showbackground": true, "ticks": "", "zerolinecolor": "white"}, "yaxis": {"backgroundcolor": "#E5ECF6", "gridcolor": "white", "gridwidth": 2, "linecolor": "white", "showbackground": true, "ticks": "", "zerolinecolor": "white"}, "zaxis": {"backgroundcolor": "#E5ECF6", "gridcolor": "white", "gridwidth": 2, "linecolor": "white", "showbackground": true, "ticks": "", "zerolinecolor": "white"}}, "shapedefaults": {"line": {"color": "#2a3f5f"}}, "ternary": {"aaxis": {"gridcolor": "white", "linecolor": "white", "ticks": ""}, "baxis": {"gridcolor": "white", "linecolor": "white", "ticks": ""}, "bgcolor": "#E5ECF6", "caxis": {"gridcolor": "white", "linecolor": "white", "ticks": ""}}, "title": {"x": 0.05}, "xaxis": {"automargin": true, "gridcolor": "white", "linecolor": "white", "ticks": "", "title": {"standoff": 15}, "zerolinecolor": "white", "zerolinewidth": 2}, "yaxis": {"automargin": true, "gridcolor": "white", "linecolor": "white", "ticks": "", "title": {"standoff": 15}, "zerolinecolor": "white", "zerolinewidth": 2}}}},                        {"responsive": true}                    ).then(function(){
                            
var gd = document.getElementById('8a08ef42-ef66-496f-88ae-baa83aaa844e');
var x = new MutationObserver(function (mutations, observer) {{
        var display = window.getComputedStyle(gd).display;
        if (!display || display === 'none') {{
            console.log([gd, 'removed!']);
            Plotly.purge(gd);
            observer.disconnect();
        }}
}});

// Listen for the removal of the full notebook cells
var notebookContainer = gd.closest('#notebook-container');
if (notebookContainer) {{
    x.observe(notebookContainer, {childList: true});
}}

// Listen for the clearing of the current output cell
var outputEl = gd.closest('.output');
if (outputEl) {{
    x.observe(outputEl, {childList: true});
}}

                        })                };                });            </script>        </div>

The included classes offer several methods that can be overridden in subclasses to customize what to mine and how to visualize it. See the chapter for details.

Here are all the classes defined in this chapter:


![](PICS/ChangeCounter-synopsis-1.svg)



## Mining Change Histories

The history of any software project is a history of change. Any nontrivial project thus comes with a _version database_ to organize and track changes; and possibly also with an [issue database](Tracking.ipynb) to organize and track issues.

Over time, these databases hold plenty of information about the project: _Who changed what, when, and why?_ This information can be _mined_ from existing databases and _analyzed_ to answer questions such as

* Which parts in my project were most frequently or recently changed?
* How many files does the average change touch?
* Where in my project were the most bugs fixed?

To answer such questions, we can _mine_ change and bug histories for past changes and fixes. This involves digging through version databases such as `git` and [issue trackers such as RedMine or Bugzilla](Tracking.ipynb) and extracting all their information. Fortunately for us, there is ready-made infrastructure for some of this. 

## Mining with PyDriller

[PyDriller](https://pydriller.readthedocs.io/) is a Python package for mining change histories. Its `RepositoryMining` class takes a `git` version repository and allows to access all the individual changes ("modifications"), together with committers, affected files, commit messages, and more.

In [ ]:
from pydriller import RepositoryMining  # https://pydriller.readthedocs.io/
from pydriller.domain.commit import Commit
from pydriller.domain.commit import Modification

To use `RepositoryMining`, we need to pass it 
* the URL of a `git` repository; or
* the directory name where a cloned `git` repository can be found.

In general, cloning a `git` repository locally (with `git clone URL`) and then analyzing it locally will be faster and require less network resources.

Let us apply `RepositoryMining` on the repository of this book. The function `current_repo()` returns the directory in which a `.git` subdirectory is stored – that is, the root of a cloned `git` repository.

In [ ]:
import os
import sys

In [ ]:
from typing import Sequence, Any, Callable, Optional, Type, Tuple, Any
from typing import Dict, Union, Set, List, FrozenSet, cast

In [ ]:
def current_repo() -> Optional[str]:
    path = os.getcwd()
    while True:
        if os.path.exists(os.path.join(path, '.git')):
            return os.path.normpath(path)

        # Go one level up
        new_path = os.path.normpath(os.path.join(path, '..'))
        if new_path != path:
            path = new_path
        else:
            return None

    return None

In [ ]:
current_repo()

This gives us a repository miner for the book:

In [ ]:
book_miner = RepositoryMining(current_repo())

You can also specify a URL instead, but this will access the repository via the network and generally be much slower.

In [ ]:
DEBUGGINGBOOK_REMOTE_REPO = 'https://github.com/uds-se/debuggingbook.git'
# book_miner = RepositoryMining(DEBUGGINGBOOK_REMOTE_REPO)

In [ ]:
# ignore
if 'CI' in os.environ:
    # In continuous integration, .git is shallow;
    # so we access the book repo via its URL
    book_miner = RepositoryMining(DEBUGGINGBOOK_REMOTE_REPO)

`traverse_commits()` is a generator that returns one commit after another. Let us fetch the very first commit made to the book:

In [ ]:
book_commits = book_miner.traverse_commits()
book_first_commit = next(book_commits)

Each commit has a number of attributes telling us more about the commit.

In [ ]:
[attr for attr in dir(book_first_commit) if not attr.startswith('_')]

For instance, the `msg` attribute lets us know about the commit message:

In [ ]:
book_first_commit.msg

whereas the `author` attribute gets us the name and email of the person who made the commit:

In [ ]:
[attr for attr in dir(book_first_commit.author) if not attr.startswith('_')]

In [ ]:
book_first_commit.author.name, book_first_commit.author.email

A commit consists of multiple _modifications_ to possibly multiple files. The commit `modifications` attribute returns a list of modifications.

In [ ]:
book_first_commit.modifications

For each modification, we can retrieve the files involved as well as several statistics:

In [ ]:
[attr for attr in dir(book_first_commit.modifications[0]) if not attr.startswith('_')]

Let us see which file was created with this modification:

In [ ]:
book_first_commit.modifications[0].new_path

The `source_code` attribute holds the entire file contents after the modification.

In [ ]:
print(book_first_commit.modifications[0].source_code)

We see that the `debuggingbook` project started with a very simple commit, namely the addition of an (almost empty) `README.md` file.

The attribute `source_code_before` holds the previous source code. We see that it is `None` – the file was just created.

In [ ]:
print(book_first_commit.modifications[0].source_code_before)

Let us have a look at the _second_ commit. We see that it is much more substantial already.

In [ ]:
book_second_commit = next(book_commits)

In [ ]:
[m.new_path for m in book_second_commit.modifications]

We fetch the modification for the `README.md` file:

In [ ]:
readme_modification = [m for m in book_second_commit.modifications if m.new_path == 'README.md'][0]

The `source_code_before` attribute holds the previous version (which we already have seen):

In [ ]:
print(readme_modification.source_code_before)

The `source_code` attribute holds the new version – now a complete "README" file. (Compare this first version to the [current README text](index.ipynb).)

In [ ]:
print(readme_modification.source_code[:400])

The `diff` attribute holds the differences between the old and the new version.

In [ ]:
print(readme_modification.diff[:100])

The `diff_parsed` attribute even lists added and deleted lines:

In [ ]:
readme_modification.diff_parsed['added'][:10]

With all this information, we can track all commits and modifications and establish statistics over which files were changed (and possibly even fixed) most. This is what we will do in the next section.

## Counting Changes

We start with a simple `ChangeCounter` class that, given a repository, counts for each file how frequently it was changed.

We represent file names as _nodes_ – a tuple $(f_1, f_2, ..., f_n)$ that denotes a _hierarchy_: Each $f_i$ is a directory holding $f_{i+1}$, with $f_n$ being the actual file. Here is what this notebook looks as a node:

In [ ]:
tuple('debuggingbook/notebooks/ChangeCounter.ipynb'.split('/'))

In [ ]:
Node = Tuple

The constructor takes the repository to be analyzed and sets the internal counters.

In [ ]:
from collections import defaultdict

In [ ]:
class ChangeCounter:
    """Count the number of changes for a repository."""

    def __init__(self, repo: str, *, 
                 filter: Optional[Callable[[Commit], bool]] = None, 
                 log: bool = False, 
                 **kwargs: Any) -> None:
        """
        Constructor.
        `repo` is a git repository (as URL or directory).
        `filter` is a predicate that takes a modification and returns True 
        if it should be considered (default: consider all).
        `log` turns on logging if set.
        `kwargs` are passed to the `RepositoryMining()` constructor.
        """
        self.repo = repo
        self.log = log

        if filter is None:
            def filter(m: Modification) -> bool:
                return True
        assert filter is not None

        self.filter = filter

        # A node is an tuple (f_1, f_2, f_3, ..., f_n) denoting
        # a folder f_1 holding a folder f_2 ... holding a file f_n.

        # Mapping node -> #of changes
        self.changes: Dict[Node, int] = defaultdict(int)

        # Mapping node -> list of commit messages
        self.messages: Dict[Node, List[str]] = defaultdict(list)

        # Mapping node -> last size seen
        self.sizes: Dict[Node, Union[int, float]] = {}

        self.mine(**kwargs)

The method `mine()` does all the heavy lifting of mining. It retrieves all commits and all modifications from the repository, passing the modifications through the `update_stats()` method.

In [ ]:
class ChangeCounter(ChangeCounter):
    def mine(self, **kwargs: Any) -> None:
        """Gather data from repository. To be extended in subclasses."""
        miner = RepositoryMining(self.repo, **kwargs)

        for commit in miner.traverse_commits():
            for m in commit.modifications:
                m.committer = commit.committer
                m.committer_date = commit.committer_date
                m.msg = commit.msg

                if self.include(m):
                    self.update_stats(m)

The `include()` method allows to filter modifications. For simplicity, we copy the most relevant attributes of the commit over to the modification, such that the filter can access them, too.

In [ ]:
class ChangeCounter(ChangeCounter):
    def include(self, m: Modification) -> bool:
        """
        Return True if the modification `m` should be included
        (default: the `filter` predicate given to the constructor).
        To be overloaded in subclasses.
        """
        return self.filter(m)

For each such node, `update_stats()` then invokes `update_size()`, `update_changes()`, and `update_elems()`.

In [ ]:
class ChangeCounter(ChangeCounter):
    def update_stats(self, m: Modification) -> None:
        """
        Update counters with modification `m`.
        Can be extended in subclasses.
        """
        if not m.new_path:
            return

        node = tuple(m.new_path.split('/'))

        self.update_size(node, len(m.source_code) if m.source_code else 0)
        self.update_changes(node, m.msg)

        self.update_elems(node, m)

`update_size()` simply saves the last size of the item being modified. Since we progress from first to last commit, this reflects the size of the newest version.

In [ ]:
class ChangeCounter(ChangeCounter):
    def update_size(self, node: Tuple, size: int) -> None:
        """
        Update counters for `node` with `size`.
        Can be extended in subclasses.
        """
        self.sizes[node] = size

`update_changes()` increases the counter `changes` for the given node `node`, and adds the current commit message `commit_msg` to its list. This makes

* `size` a mapping of nodes to their size
* `changes` a mapping of nodes to the number of changes they have seen
* `commit_msg` a mapping of nodes to the list of commit messages that have affected them.

In [ ]:
class ChangeCounter(ChangeCounter):
    def update_changes(self, node: Tuple, commit_msg: str) -> None:
        """
        Update stats for `node` changed with `commit_msg`.
        Can be extended in subclasses.
        """
        self.changes[node] += 1

        self.messages[node].append(commit_msg)

The `update_elems()` method is reserved for later use, when we go and count fine-grained changes.

In [ ]:
class ChangeCounter(ChangeCounter):
    def update_elems(self, node: Tuple, m: Modification) -> None:
        """
        Update counters for subelements of `node` with modification `m`.
        To be defined in subclasses.
        """
        pass

Let us put `ChangeCounter` to action – on the current (debuggingbook) repository.

In [ ]:
DEBUGGINGBOOK_REPO = current_repo()

In [ ]:
DEBUGGINGBOOK_REPO

In [ ]:
# ignore
if 'CI' in os.environ:
    # As above: We have to access the repo remotely
    DEBUGGINGBOOK_REPO = DEBUGGINGBOOK_REMOTE_REPO

The function `debuggingbook_change_counter` instantiates a `ChangeCounter` class (or any subclass) with the debuggingbook repository, mining all the counters as listed above. Since mining _all_ history takes quite some time, its parameter `start_date` allows to set a starting date (default: March 1, 2021); changes before that date will be ignored.

In [ ]:
from datetime import datetime

In [ ]:
def debuggingbook_change_counter(cls: Type, 
                                 start_date: datetime = datetime(2021, 3, 1)) \
                                 -> Any:
    """
    Instantiate a ChangeCounter (sub)class `cls` with the debuggingbook repo.
    Only mines changes after `start_date` (default: March 1, 2021)
    """

    def filter(m: Modification) -> bool:
        """
        Do not include
        * the `docs/` directory; it only holds generated Web pages
        * the `synopsis` pictures; these are all generated
        * the `notebooks/shared/` package; this is infrastructure
        """
        return (m.new_path and
                not m.new_path.startswith('docs/') and
                not m.new_path.startswith('notebooks/shared/') and
                not '-synopsis-' in m.new_path)

    return cls(DEBUGGINGBOOK_REPO, filter=filter, since=start_date)

Let us set `change_counter` to this `ChangeCounter` instance. This can take a few minutes.

In [ ]:
from Timer import Timer

In [ ]:
with Timer() as t:
    change_counter = debuggingbook_change_counter(ChangeCounter)

t.elapsed_time()

The attribute `changes` of our `ChangeCounter` now is a mapping of nodes to the respective number of changes. Here are the first 10 entries:

In [ ]:
list(change_counter.changes.keys())[:10]

This is the number of changes to the `Chapters.makefile` file which lists the book chapters:

In [ ]:
change_counter.changes[('Chapters.makefile',)]

The `messages` attribute holds all the messages:

In [ ]:
change_counter.messages[('Chapters.makefile',)]

In [ ]:
for node in change_counter.changes:
    assert len(change_counter.messages[node]) == change_counter.changes[node]

The `sizes` attribute holds the final size:

In [ ]:
change_counter.sizes[('Chapters.makefile',)]

## Visualizing Past Changes

To explore the number of changes across all project files, we visualize them as a _tree map_. A tree map visualizes hierarchical data using nested rectangles. In our visualization, each directory is shown as a rectangle containing smaller rectangles. The _size_ of a rectangle is relative to its size (in bytes); and the _color_ of a rectangle is relative to the number of changes it has seen.

We use the [easyplotly](https://github.com/mwouts/easyplotly) package to easily create a treemap.

In [ ]:
import easyplotly as ep
import plotly.graph_objects as go

In [ ]:
import math

The method `map_node_sizes()` returns a size for the node – any number will do. By default, we use a logarithmic scale, such that smaller files are not totally visually eclipsed by larger files.

In [ ]:
class ChangeCounter(ChangeCounter):
    def map_node_sizes(self, scale: str = 'log') -> \
        Dict[Node, Union[int, float]]:
        """
        Return a mapping of nodes to sizes.
        Can be overloaded in subclasses.
        """

        if scale == 'log':
            # Default: use log scale
            return {node: math.log(size+1) 
                    for node, size in self.sizes.items()}

        elif scale == 'sqrt':
            # Alternative: use sqrt size
            return {node: math.sqrt(size)
                    for node, size in self.sizes.items()}

        elif scale == 'abs':
            # Alternative: use absolute size
            return self.sizes

        else:
            raise ValueError(f"Unknown scale: {scale}; "
                             f"available scaling modes are [log, sqrt, abs]")

The method `map_node_color()` returns a color for the node – again, as a number. The smallest and largest numbers returned indicate beginning and end in the given color scale, respectively.

In [ ]:
class ChangeCounter(ChangeCounter):
    def map_node_color(self, node: Node) -> Optional[int]:
        """
        Return a color of the node, as a number.
        Can be overloaded in subclasses.
        """
        return self.changes.get(node)

The method `map_node_text()` shows a text to be displayed in the rectangle; we set this to the number of changes.

In [ ]:
class ChangeCounter(ChangeCounter):
    def map_node_text(self, node: Node) -> Optional[str]:
        """
        Return the text to be shown for the node (default: #changes).
        Can be overloaded in subclasses.
        """
        change = self.changes.get(node)
        return str(change) if change is not None else None

The methods `map_hoverinfo()` and `map_colorscale()` set additional map parameters. For details, see the [easyplotly](https://github.com/mwouts/easyplotly) documentation.

In [ ]:
class ChangeCounter(ChangeCounter):
    def map_hoverinfo(self) -> str:
        """
        Return the text to be shown when hovering over a node.
        To be overloaded in subclasses.
        """
        return 'label+text'

    def map_colorscale(self) -> str:
        """
        Return the colorscale for the map. To be overloaded in subclasses.
        """
        return 'YlOrRd'

With all this, the `map()` function creates a tree map of the repository, using the  [easyplotly](https://github.com/mwouts/easyplotly) `Treemap` constructor.

In [ ]:
class ChangeCounter(ChangeCounter):
    def map(self) -> go.Figure:
        """Produce an interactive tree map of the repository."""
        treemap = ep.Treemap(
                     self.map_node_sizes(),
                     text=self.map_node_text,
                     hoverinfo=self.map_hoverinfo(),
                     marker_colors=self.map_node_color,
                     marker_colorscale=self.map_colorscale(),
                     root_label=self.repo,
                     branchvalues='total'
                    )

        fig = go.Figure(treemap)
        fig.update_layout(margin=dict(l=0, r=0, t=30, b=0))

        return fig

This is what the tree map for `debuggingbook` looks like. 

* Click on any rectangle to enlarge it.
* Click outside of the rectangle to return to a wider view.
* Hover over a rectangle to get further information.

In [ ]:
change_counter = debuggingbook_change_counter(ChangeCounter)

In [ ]:
change_counter.map()

We can easily identify the most frequently changed files:

In [ ]:
sorted(change_counter.changes.items(), key=lambda kv: kv[1], reverse=True)[:4]

In [ ]:
# ignore
all_notebooks = [node for node in change_counter.changes.keys()
                 if len(node) == 2 and node[1].endswith('.ipynb')]
all_notebooks.sort(key=lambda node: change_counter.changes[node], reverse=True)

In [ ]:
from bookutils import quiz

In [ ]:
quiz("Which two notebooks have seen the most changes over time?",
    [
        f"`{all_notebooks[0][1].split('.')[0]}`",
        f"`{all_notebooks[3][1].split('.')[0]}`",
        f"`{all_notebooks[1][1].split('.')[0]}`",
        f"`{all_notebooks[2][1].split('.')[0]}`",
    ], '[1234 % 3, 3702 / 1234]')

Indeed, these two are the two most frequently recently changed notebooks:

In [ ]:
all_notebooks[0][1].split('.')[0], all_notebooks[1][1].split('.')[0]

## Counting Past Fixes

Knowing which files have been changed most is useful in debugging, because any change increases the chance to introduce a new bug. Even more important, however, is the question of how frequently a file was _fixed_ in the past, as this is an important indicator for its bug-proneness.

(One may think that fixing several bugs _reduces_ the number of bugs, but unfortunately, a file which has seen several fixes in the past is likely to see fixes in the future, too. This is because the bug-proneness of a software component very much depends on the requirements it has to fulfill, and if these requirements are unclear, complex, or frequently change, this translates into many fixes.)

How can we tell _changes_ from _fixes_? 

* One indicator is _commit messages_:
  If they refer to "bugs" or "fixes", then the change is a fix.
* Another indicator is _bug numbers_:
  If a commit message contains an issue number from an associated issue database, then we can make use of the issue referred to.
    * The issue database may provide us with additional information about the bug, such as its severity, how many people it was assigned to, how long it took to fix it, and more.
* A final indicator is _time_:
  If a developer first committed a change and in the same time frame marked an issue as "resolved", then it is likely that the two refer to each other.

The way these two are linked very much depends on the project – and the discipline of developers as it comes to change messages. _Branches_ and _merges_ bring additional challenges.

For the `debuggingbook` project, identifying fixes is easy. The discipline is that if a change fixes a bug, it is prefixed with `Fix:`. We can use this to introduce a `FixCounter` class specific to our `debuggingbook` project.

In [ ]:
class FixCounter(ChangeCounter):
    """
    Count the fixes for files in the repository.
    Fixes are all commits whose message starts with the word 'Fix: '
    """

    def include(self, m: Modification) -> bool:
        """Include all modifications whose commit messages start with 'Fix:'"""
        return super().include(m) and m and m.msg.startswith("Fix:")

As a twist to our default `ChangeCounter` class, we include the "fix" messages in the tree map rectangles.

In [ ]:
class FixCounter(FixCounter):
    def map_node_text(self, node: Node) -> str:
        return "<br>".join(self.messages.get(node, []))

    def map_hoverinfo(self) -> str:
        return 'label'

This is the tree map showing fixes. We see that 
* only those components that actually have seen a fix are shown; and
* the fix distribution differs from the change distribution.

In [ ]:
fix_counter = debuggingbook_change_counter(FixCounter)

In [ ]:
fix_counter.map()

If you want to take a look at the _entire_ fix history, pass an early `start_date` argument to `debugging_change_counter()`. This takes a few extra minutes, so we make it optional:

In [ ]:
# fix_counter = debuggingbook_change_counter(FixCounter, start_date=datetime(1999, 1, 1))
# fix_counter.map()

## Counting Fine-Grained Changes

In programming projects, individual files typically consist of _smaller units_ such as functions, classes, and methods. We want to determine which of these _units_ are frequently changed (and fixed). For this, we need to _break down_ individual files into smaller parts, and then determine which of these parts would be affected by a change.

### Mapping Elements to Locations

Our first task is a simple means to split a (programming) file into smaller parts, each with their own locations. First, we need to know what kind of content a file contains. To this end, we use the Python [magic](https://github.com/ahupp/python-magic) package. (The "magic" in the name does not refer to some "magic" functionality, but to the practice of having files start with "magic" bytes that indicate their type.)

In [ ]:
import magic

The `magic` package easily guesses that a file contains C code:

In [ ]:
magic.from_buffer('''
#include <stdio.h>

int main(int argc, char *argv[]) {
    printf("Hello, world!\n")
}
''')

It also works well for Python code:

In [ ]:
magic.from_buffer('''
def foo():
    print("Hello, world!")
''')

Jupyter Notebooks, however, are identified as `JSON` data or `SGML` documents:

In [ ]:
magic.from_buffer(open(os.path.join(current_repo(),   # type: ignore
                                    'notebooks',
                                    'Assertions.ipynb')).read())

We define a set of _delimiters_ for these languages which use _regular expressions_ to identify
* the _language_ (matching the `magic` output)
* the _beginning of a unit_, and
* the _end_ of a unit,

For Python, for instance, any line starting with `def` or `class` denotes the start of some unit; any line starting with something else denotes the end of a unit. For Jupyter, the delimiters do the same, yet encoded into JSON. The definitions for C are likely to work for a wide range of languages that all use `{` and `}` to delimit units.

In [ ]:
import re

In [ ]:
from typing import Pattern

In [ ]:
DELIMITERS: List[Tuple[Pattern, Pattern, Pattern]] = [
    (
        # Python
        re.compile(r'^python.*'),

        # Beginning of element
        re.compile(r'^(async\s+)?(def|class)\s+(?P<name>\w+)\W.*'),

        # End of element
        re.compile(r'^[^#\s]')
    ),
    (
        # Jupyter Notebooks
        re.compile(r'^(json|exported sgml|jupyter).*'),
        re.compile(r'^\s+"(async\s+)?(def|class)\s+(?P<name>\w+)\W'),
        re.compile(r'^(\s+"[^#\s\\]|\s+\])')
    ),
    (
        # C source code (actually, any { }-delimited language)
        re.compile(r'^(c |c\+\+|c#|java|perl|php).*'),
        re.compile(r'^[^\s].*\s+(?P<name>\w+)\s*[({].*'),
        re.compile(r'^[}]')
    )
]

The function `rxdelim()` returns suitable delimiters for a given content, using `DELIMITERS`.

In [ ]:
def rxdelim(content: str) -> Tuple[Optional[Pattern], Optional[Pattern]]:
    """
    Return suitable begin and end delimiters for the content `content`.
    If no matching delimiters are found, return `None, None`.
    """
    tp = magic.from_buffer(content).lower()
    for rxtp, rxbegin, rxend in DELIMITERS:
        if rxtp.match(tp):
            return rxbegin, rxend

    return None, None

The function `elem_mapping()` returns a list of the individual elements as found in the file, indexed by line numbers (starting with 1).

In [ ]:
Mapping = List[Optional[str]]

In [ ]:
def elem_mapping(content: str, log: bool = False) -> Mapping:
    """Return a list of the elements in `content`, indexed by line number."""
    rxbegin, rxend = rxdelim(content)
    if rxbegin is None:
        return []
    if rxend is None:
        return []

    mapping: List[Optional[str]] = [None]
    current_elem = None
    lineno = 0

    for line in content.split('\n'):
        lineno += 1

        match = rxbegin.match(line)
        if match:
            current_elem = match.group('name')
        elif rxend.match(line):
            current_elem = None

        mapping.append(current_elem)

        if log:
            print(f"{lineno:3} {str(current_elem):15} {line}")

    return mapping

Here is an example of how `elem_mapping()` works. During execution (with `log` set to `True`), we already see the elements associated with individual line numbers.

In [ ]:
some_c_source = """
#include <stdio.h>

int foo(int x) {
    return x;
}

struct bar {
    int x, y;
}

int main(int argc, char *argv[]) {
    return foo(argc);
}

"""
some_c_mapping = elem_mapping(some_c_source, log=True)

In the actual mapping, we can access the individual units for any line number:

In [ ]:
some_c_mapping[1], some_c_mapping[8]

Here's how this works for Python:

In [ ]:
some_python_source = """
def foo(x):
    return x

class bar(blue):
    x = 25
    def f(x):
        return 26

def main(argc):
    return foo(argc)

"""
some_python_mapping = elem_mapping(some_python_source, log=True)

In [ ]:
# some_jupyter_source = open("Slicer.ipynb").read()
# some_jupyter_mapping = elem_mapping(some_jupyter_source, log=False)

### Determining Changed Elements

Using a mapping from `elem_mapping()`, we can determine which elements are affected by a change. The `changed_elems_by_mapping()` function returns the set of affected elements.

In [ ]:
def changed_elems_by_mapping(mapping: Mapping, start: int, length: int = 0) -> Set[str]:
    """
    Within `mapping`, return the set of elements affected by a change
    starting in line `start` and extending over `length` additional lines.
    """
    elems = set()
    for line in range(start, start + length + 1):
        if line < len(mapping) and mapping[line]:
            elem = mapping[line]
            assert elem is not None
            elems.add(elem)

    return elems

Here's an example of `changed_elems_by_mapping()`, applied to the Python content, above:

In [ ]:
changed_elems_by_mapping(some_python_mapping, start=2, length=4)

The function `elem_size()` returns the size of an element (say, a function).

In [ ]:
def elem_size(elem: str, source: str) -> int:
    """Within `source`, return the size of `elem`"""
    source_lines = [''] + source.split('\n')
    size = 0
    mapping = elem_mapping(source)

    for line_no in range(len(mapping)):
        if mapping[line_no] == elem or mapping[line_no] is elem:
            size += len(source_lines[line_no] + '\n')

    return size

In [ ]:
elem_size('foo', some_python_source)

In [ ]:
assert sum(elem_size(name, some_python_source) 
           for name in ['foo', 'bar', 'main']) == len(some_python_source)

Given an old version and a new version of a (text) file, we can use the `diff_match_patch` module to determine differences, and from these the affected lines:

In [ ]:
from ChangeDebugger import diff  # minor dependency

In [ ]:
from diff_match_patch import diff_match_patch

In [ ]:
def changed_elems(old_source: str, new_source: str) -> Set[str]:
    """Determine the elements affected by the change from `old_source` to `new_source`"""
    patches = diff(old_source, new_source)

    old_mapping = elem_mapping(old_source)
    new_mapping = elem_mapping(new_source)

    elems = set()

    for patch in patches:
        old_start_line = patch.start1 + 1
        new_start_line = patch.start2 + 1

        for (op, data) in patch.diffs:
            length = data.count('\n')

            if op == diff_match_patch.DIFF_INSERT:
                elems |= changed_elems_by_mapping(old_mapping, old_start_line)
                elems |= changed_elems_by_mapping(new_mapping, new_start_line, length)
            elif op == diff_match_patch.DIFF_DELETE:
                elems |= changed_elems_by_mapping(old_mapping, old_start_line, length)
                elems |= changed_elems_by_mapping(new_mapping, new_start_line)

            old_start_line += length
            new_start_line += length

    return elems

Here is how `changed_elems()` works. We define a "new" version of `some_python_source`:

In [ ]:
some_new_python_source = """
def foo(y):
    return y

class qux(blue):
    x = 25
    def f(x):
        return 26

def main(argc):
    return foo(argc)

"""

In [ ]:
changed_elems(some_python_source, some_new_python_source)

Note that the list of changed elements includes added as well as deleted elements.

### Putting it all Together

We introduce a class `FineChangeCounter` that, like `ChangeCounter`, counts changes for individual files; however, `FineChangeCounter` adds additional nodes for all elements affected by the change. For a file consisting of multiple elements, this has the same effect as if the file were a directory, and the elements were all contained as individual files in this directory.

In [ ]:
class FineChangeCounter(ChangeCounter):
    """Count the changes for files in the repository and their elements"""

    def update_elems(self, node: Node, m: Modification) -> None:
        old_source = m.source_code_before if m.source_code_before else ""
        new_source = m.source_code if m.source_code else ""

        for elem in changed_elems(old_source, new_source):
            elem_node = node + (elem,)

            self.update_size(elem_node, elem_size(elem, new_source))
            self.update_changes(elem_node, m.msg)

Retrieving fine-grained changes takes a bit more time, since all files have to be parsed...

In [ ]:
with Timer() as t:
    fine_change_counter = debuggingbook_change_counter(FineChangeCounter)

t.elapsed_time()

... but the result is very much worth it. We can now zoom into individual files and compare the change counts for the individual functions.

In [ ]:
fine_change_counter.map()

Like before, we can access the most frequently changed elements, This is the most frequently changed item in the book:

In [ ]:
elem_nodes = [node for node in fine_change_counter.changes.keys()
              if len(node) == 3 and node[1].endswith('.ipynb')]
elem_nodes.sort(key=lambda node: fine_change_counter.changes[node], reverse=True)
[(node, fine_change_counter.changes[node]) for node in elem_nodes[:1]]

In [ ]:
from bookutils import quiz

In [ ]:
quiz("Which is the _second_ most changed element?",
    [
        f"`{elem_nodes[3][2]}` in `{elem_nodes[3][1].split('.ipynb')[0]}`",
        f"`{elem_nodes[1][2]}` in `{elem_nodes[1][1].split('.ipynb')[0]}`",
        f"`{elem_nodes[2][2]}` in `{elem_nodes[2][1].split('.ipynb')[0]}`",
        f"`{elem_nodes[0][2]}` in `{elem_nodes[0][1].split('.ipynb')[0]}`",
    ], '1975308642 // 987654321')

Indeed, here come the top five most frequently changed elements:

In [ ]:
[(node, fine_change_counter.changes[node]) for node in elem_nodes[:5]]

Now it is time to apply these tools on your own projects. Which are the most frequently changed (and fixed) elements? Why is that so? What can you do to improve things? All these are consequences of debugging – to help having fewer bugs in the future!

## Synopsis

This chapter provides two classes `ChangeCounter` and `FineChangeCounter` that allow to mine and visualize the distribution of changes in a given `git` repository.

`ChangeCounter` is initialized as

```python
change_counter = ChangeCounter(repository)
```
where `repository` is either 

* a _directory_ containing a `git` clone (i.e., it contains a `.git` directory)
* the URL of a `git` repository.

Additional arguments are being passed to the underlying `RepositoryMining` class from the [PyDriller](https://pydriller.readthedocs.io/) Python package. A `filter` keyword argument, if given, is a predicate that takes a modification (from PyDriller) and returns True if it should be included.

In a change counter, all elements in the repository are represented as _nodes_ – tuples $(f_1, f_2, ..., f_n)$ that denote a _hierarchy_: Each $f_i$ is a directory holding $f_{i+1}$, with $f_n$ being the actual file.

A `change_counter` provides a number of attributes. `changes` is a mapping of nodes to the number of changes in that node:

In [ ]:
change_counter.changes[('README.md',)]

The `messages` attribute holds all commit messages related to that node:

In [ ]:
change_counter.messages[('README.md',)]

The `sizes` attribute holds the (last) size of the respective element:

In [ ]:
change_counter.sizes[('README.md',)]

`FineChangeCounter` acts like `ChangeCounter`, but also retrieves statistics for elements _within_ the respective files; it has been tested for C, Python, and Jupyter Notebooks and should provide sufficient results for programming languages with similar syntax.

The `map()` method of `ChangeCounter` and `FineChangeCounter` produces an interactive tree map that allows to explore the elements of a repository. The redder (darker) a rectangle, the more changes it has seen; the larger a rectangle, the larger its size in bytes.

In [ ]:
fine_change_counter.map()

The included classes offer several methods that can be overridden in subclasses to customize what to mine and how to visualize it. See the chapter for details.

Here are all the classes defined in this chapter:

In [ ]:
# ignore
from ClassDiagram import display_class_hierarchy

In [ ]:
# ignore
display_class_hierarchy([FineChangeCounter, FixCounter],
                        public_methods=[
                            ChangeCounter.__init__,
                            ChangeCounter.map
                        ],
                        project='debuggingbook')

## Lessons Learned

* We can easily _mine_ past changes and map these to individual files and elements
* This information can be helpful in guiding the debugging and development process
* Counting _fixes_ needs to be customized to the conventions used in the project at hand

## Background

Before you venture out building your own development analytics, you may want to check out what's already there and available. This [overview at Livable Software](https://livablesoftware.com/tools-mine-analyze-github-git-software-data/) gives a great overview of platforms and tools for mining development data. One of the most ambitious ones is [GrimoireLab](https://chaoss.github.io/grimoirelab/), set to mine data from pretty much _any_ open source repository.

\todo{To be added}

## Exercises


### Exercise 1: Fine-Grained Fixes

Construct a class `FineFixCounter` that combines fine-grained counting from `FineChangeCounter` with the fix-counting abilities from `FixCounter`. Visualize the treemap.

**Solution.** We can use multiple inheritance to combine both. We inherit from `FixCounter` first such that we can see the fix messages.

In [ ]:
class FineFixCounter(FixCounter, FineChangeCounter):
    pass

Try things out for yourself:

In [ ]:
# fine_fix_counter = debuggingbook_change_counter(FineFixCounter, start_date=datetime(1999, 1, 1))
# fine_fix_counter.map()